In [ ]:
# Install necessary dependencies
!pip install streamlit opencv-python-headless pyngrok
!pip install ultralytics
!pip install 'git+https://github.com/facebookresearch/pytorchvideo.git'
!pip install moviepy

# Write the Streamlit app code to a file
app_code = """
import streamlit as st
import os
import shutil
import subprocess
import torch
import cv2
from PIL import Image
from ultralytics import YOLO
from slowfast.config.defaults import get_cfg
from slowfast.models import build_model
from torchvision import transforms

# YOLOv8 Model Initialization
yolo_model = YOLO('yolov8n.pt')

# SlowFast Model Initialization
cfg = get_cfg()
cfg.merge_from_file('configs/Kinetics/SLOWFAST_8x8_R50.yaml')
cfg.NUM_GPUS = 1
cfg.TEST.CHECKPOINT_FILE_PATH = "SLOWFAST_8x8_R50.pyth"
slowfast_model = build_model(cfg)

# Load the SlowFast weights
checkpoint = torch.load(cfg.TEST.CHECKPOINT_FILE_PATH, map_location="cpu")
slowfast_model.load_state_dict(checkpoint["model_state"])
slowfast_model.eval()

def process_video_with_yolo(video_path):
    output_dir = 'runs/detect/exp'
    crops_dir = os.path.join(output_dir, 'crops', 'cell phone')
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)  # Clear previous runs

    command = ['python', '-m', 'ultralytics.run', 'detect', '--weights', 'yolov8n.pt', '--source', video_path,
               '--classes', '67', '--conf', '0.10', '--save-txt', '--save-conf', '--project', 'runs/detect', '--exist-ok', '--save-crop']
    result = subprocess.run(command, capture_output=True, text=True)

    detected_frames = []
    if os.path.exists(crops_dir):
        for file in os.listdir(crops_dir):
            if file.endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(crops_dir, file)
                img = Image.open(img_path)
                detected_frames.append(img)
    return detected_frames

def detect_behavior_with_slowfast(frames_dir):
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225]),
    ])
    video_tensor = []
    for frame_file in os.listdir(frames_dir):
        if frame_file.endswith(('.jpg', '.jpeg', '.png')):
            frame = Image.open(os.path.join(frames_dir, frame_file))
            frame_tensor = transform(frame).unsqueeze(0)
            video_tensor.append(frame_tensor)

    if len(video_tensor) == 0:
        return False  # No frames to process

    video_tensor = torch.cat(video_tensor, dim=0).unsqueeze(0)  # Simulate batch of size 1
    output = slowfast_model(video_tensor)

    # Simplified logic for demonstration (real applications require better output interpretation)
    behavior_detected = output.max().item() > 0.5  # Dummy threshold
    return behavior_detected

def create_collage(frames, cols=5):
    if len(frames) == 0:
        return None
    frame_width, frame_height = frames[0].size
    rows = (len(frames) + cols - 1) // cols
    collage_width = cols * frame_width
    collage_height = rows * frame_height
    collage = Image.new('RGB', (collage_width, collage_height))
    for idx, frame in enumerate(frames):
        row = idx // cols
        col = idx % cols
        collage.paste(frame, (col * frame_width, row * frame_height))
    return collage

# Streamlit app
st.title('Driver Safety Detection: Cell Phones & Head Turning')
uploaded_file = st.file_uploader('Upload a video file', type=['mp4', 'avi', 'mov'])
if uploaded_file is not None:
    video_path = f'uploaded_video.{uploaded_file.type.split("/")[1]}'
    with open(video_path, "wb") as f:
        f.write(uploaded_file.getbuffer())

    st.video(video_path)
    st.write('Running YOLOv8 inference...')
    try:
        detected_frames = process_video_with_yolo(video_path)
        st.write('YOLOv8 inference completed.')
        if detected_frames:
            st.write(f'Detected {len(detected_frames)} frames with cell phones.')
            collage_image = create_collage(detected_frames)
            if collage_image:
                st.image(collage_image, caption='Collage of detected frames')

            # Run behavior detection
            st.write('Running SlowFast behavior analysis...')
            behavior_detected = detect_behavior_with_slowfast(frames_dir='runs/detect/exp/crops/cell phone')
            if behavior_detected:
                st.write('Head turning behavior detected.')
                st.success('True Positive: Cell phone detected and head turning behavior detected.')
            else:
                st.write('No head turning behavior detected.')
                st.error('False Positive: Cell phone detected but no head turning behavior detected.')
        else:
            st.write('No cell phones detected in the video.')
            st.error('False Positive: No cell phones detected.')
    except Exception as e:
        st.write(f'Error during inference: {e}')
"""

with open("app.py", "w") as file:
    file.write(app_code)
print("app.py file has been written.")

# Start the Streamlit app in the background
import subprocess
subprocess.Popen(['streamlit', 'run', 'app.py'])

# Expose the Streamlit app using pyngrok
!wget -q -O - https://loca.lt/mytunnelpassword

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadat

In [ ]:
# Install necessary dependencies
!pip install streamlit opencv-python-headless pyngrok
!pip install ultralytics
!pip install 'git+https://github.com/facebookresearch/pytorchvideo.git'
!pip install moviepy

# Clone the SlowFast repository to ensure it's available in the file explorer
!git clone https://github.com/facebookresearch/SlowFast.git
%cd SlowFast
!pip install -r requirements.txt
%cd ..

# Write the Streamlit app code to a file
app_code = """
import streamlit as st
import os
import shutil
import subprocess
import torch
import cv2
from PIL import Image
from ultralytics import YOLO
from slowfast.config.defaults import get_cfg
from slowfast.models import build_model
from torchvision import transforms

# YOLOv8 Model Initialization
yolo_model = YOLO('yolov8n.pt')

# SlowFast Model Initialization
cfg = get_cfg()
cfg.merge_from_file('SlowFast/configs/Kinetics/SLOWFAST_8x8_R50.yaml')
cfg.NUM_GPUS = 1
cfg.TEST.CHECKPOINT_FILE_PATH = "SLOWFAST_8x8_R50.pyth"
slowfast_model = build_model(cfg)

# Load the SlowFast weights
checkpoint = torch.load(cfg.TEST.CHECKPOINT_FILE_PATH, map_location="cpu")
slowfast_model.load_state_dict(checkpoint["model_state"])
slowfast_model.eval()

def process_video_with_yolo(video_path):
    output_dir = 'runs/detect/exp'
    crops_dir = os.path.join(output_dir, 'crops', 'cell phone')
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)  # Clear previous runs

    command = ['python', '-m', 'ultralytics.run', 'detect', '--weights', 'yolov8n.pt', '--source', video_path,
               '--classes', '67', '--conf', '0.10', '--save-txt', '--save-conf', '--project', 'runs/detect', '--exist-ok', '--save-crop']
    result = subprocess.run(command, capture_output=True, text=True)

    detected_frames = []
    if os.path.exists(crops_dir):
        for file in os.listdir(crops_dir):
            if file.endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(crops_dir, file)
                img = Image.open(img_path)
                detected_frames.append(img)
    return detected_frames

def detect_behavior_with_slowfast(frames_dir):
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225]),
    ])
    video_tensor = []
    for frame_file in os.listdir(frames_dir):
        if frame_file.endswith(('.jpg', '.jpeg', '.png')):
            frame = Image.open(os.path.join(frames_dir, frame_file))
            frame_tensor = transform(frame).unsqueeze(0)
            video_tensor.append(frame_tensor)

    if len(video_tensor) == 0:
        return False  # No frames to process

    video_tensor = torch.cat(video_tensor, dim=0).unsqueeze(0)  # Simulate batch of size 1
    output = slowfast_model(video_tensor)

    # Simplified logic for demonstration (real applications require better output interpretation)
    behavior_detected = output.max().item() > 0.5  # Dummy threshold
    return behavior_detected

def create_collage(frames, cols=5):
    if len(frames) == 0:
        return None
    frame_width, frame_height = frames[0].size
    rows = (len(frames) + cols - 1) // cols
    collage_width = cols * frame_width
    collage_height = rows * frame_height
    collage = Image.new('RGB', (collage_width, collage_height))
    for idx, frame in enumerate(frames):
        row = idx // cols
        col = idx % cols
        collage.paste(frame, (col * frame_width, row * frame_height))
    return collage

# Streamlit app
st.title('Driver Safety Detection: Cell Phones & Head Turning')
uploaded_file = st.file_uploader('Upload a video file', type=['mp4', 'avi', 'mov'])
if uploaded_file is not None:
    video_path = f'uploaded_video.{uploaded_file.type.split("/")[1]}'
    with open(video_path, "wb") as f:
        f.write(uploaded_file.getbuffer())

    st.video(video_path)
    st.write('Running YOLOv8 inference...')
    try:
        detected_frames = process_video_with_yolo(video_path)
        st.write('YOLOv8 inference completed.')
        if detected_frames:
            st.write(f'Detected {len(detected_frames)} frames with cell phones.')
            collage_image = create_collage(detected_frames)
            if collage_image:
                st.image(collage_image, caption='Collage of detected frames')

            # Run behavior detection
            st.write('Running SlowFast behavior analysis...')
            behavior_detected = detect_behavior_with_slowfast(frames_dir='runs/detect/exp/crops/cell phone')
            if behavior_detected:
                st.write('Head turning behavior detected.')
                st.success('True Positive: Cell phone detected and head turning behavior detected.')
            else:
                st.write('No head turning behavior detected.')
                st.error('False Positive: Cell phone detected but no head turning behavior detected.')
        else:
            st.write('No cell phones detected in the video.')
            st.error('False Positive: No cell phones detected.')
    except Exception as e:
        st.write(f'Error during inference: {e}')
"""

with open("app.py", "w") as file:
    file.write(app_code)
print("app.py file has been written.")

# Start the Streamlit app in the background
import subprocess
subprocess.Popen(['streamlit', 'run', 'app.py'])

import requests
import time

# Wait a few seconds for the Streamlit app to start
time.sleep(5)

# Start LocalTunnel
localtunnel_process = subprocess.Popen(['lt', '--port', '8501'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Capture the LocalTunnel URL
public_url = None
for line in iter(localtunnel_process.stdout.readline, b''):
    decoded_line = line.decode().strip()
    print(decoded_line)
    if 'your url is:' in decoded_line:
        public_url = decoded_line.split(' ')[-1]
        break

if public_url:
    print(f'Streamlit app running at: {public_url}')
    # Make a request to the LocalTunnel URL with custom headers to bypass the password screen
    headers = {
        'User-Agent': 'Mozilla/5.0',
        'bypass-tunnel-reminder': '1'
    }
    response = requests.get(public_url, headers=headers)
    if response.status_code == 200:
        print(f'Access the Streamlit app at: {public_url}')
    else:
        print('Failed to bypass the password screen.')
else:
    print('Failed to create LocalTunnel.')

# Expose the Streamlit app using the provided URL
!wget -q -O - https://loca.lt/mytunnelpassword

  Cloning https://github.com/facebookresearch/pytorchvideo.git to /tmp/pip-req-build-1ewdo7pm
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/pytorchvideo.git /tmp/pip-req-build-1ewdo7pm
  Resolved https://github.com/facebookresearch/pytorchvideo.git to commit e02c17b480f4b2a70a000df4e1aaf9d45fdaefb5
  Preparing metadata (setup.py) ... done
Cloning into 'SlowFast'...
remote: Enumerating objects: 2149, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 2149 (delta 46), reused 59 (delta 35), pack-reused 2013 (from 1)
Receiving objects: 100% (2149/2149), 31.50 MiB | 26.21 MiB/s, done.
Resolving deltas: 100% (1457/1457), done.
/content/SlowFast
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
/content
app.py file has been written.


FileNotFoundError: [Errno 2] No such file or directory: 'lt'

In [ ]:

!npm install -g localtunnel

# Install necessary dependencies
!pip install streamlit opencv-python-headless pyngrok
!pip install ultralytics
!pip install 'git+https://github.com/facebookresearch/fvcore'
!pip install simplejson
!pip install -U iopath
!pip install psutil
!pip install opencv-python
!pip install torchvision
!pip install tensorboard
!pip install moviepy
!pip install pytorchvideo
!pip install 'git+https://github.com/facebookresearch/fairscale'

# Install PyAV and ffmpeg
!conda install av -c conda-forge

# Clone the SlowFast repository and install it
!git clone https://github.com/facebookresearch/slowfast.git
%cd slowfast

# Add SlowFast to PYTHONPATH
import os
os.environ['PYTHONPATH'] += ":/content/slowfast"

# Build and develop SlowFast
!python setup.py build develop

# Write the Streamlit app code to a file
app_code = """
import streamlit as st
import os
import shutil
import subprocess
import torch
import cv2
from PIL import Image
from ultralytics import YOLO
from slowfast.config.defaults import get_cfg
from slowfast.models import build_model
from torchvision import transforms

# YOLOv8 Model Initialization
yolo_model = YOLO('yolov8n.pt')

# SlowFast Model Initialization
cfg = get_cfg()
cfg.merge_from_file('slowfast/configs/Kinetics/SLOWFAST_8x8_R50.yaml')
cfg.NUM_GPUS = 1
cfg.TEST.CHECKPOINT_FILE_PATH = "SLOWFAST_8x8_R50.pyth"
slowfast_model = build_model(cfg)

# Load the SlowFast weights
checkpoint = torch.load(cfg.TEST.CHECKPOINT_FILE_PATH, map_location="cpu")
slowfast_model.load_state_dict(checkpoint["model_state"])
slowfast_model.eval()

def process_video_with_yolo(video_path):
    output_dir = 'runs/detect/exp'
    crops_dir = os.path.join(output_dir, 'crops', 'cell phone')
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)  # Clear previous runs

    command = ['python', '-m', 'ultralytics.run', 'detect', '--weights', 'yolov8n.pt', '--source', video_path,
               '--classes', '67', '--conf', '0.10', '--save-txt', '--save-conf', '--project', 'runs/detect', '--exist-ok', '--save-crop']
    result = subprocess.run(command, capture_output=True, text=True)

    detected_frames = []
    if os.path.exists(crops_dir):
        for file in os.listdir(crops_dir):
            if file.endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(crops_dir, file)
                img = Image.open(img_path)
                detected_frames.append(img)
    return detected_frames

def detect_behavior_with_slowfast(frames_dir):
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225]),
    ])
    video_tensor = []
    for frame_file in os.listdir(frames_dir):
        if frame_file.endswith(('.jpg', '.jpeg', '.png')):
            frame = Image.open(os.path.join(frames_dir, frame_file))
            frame_tensor = transform(frame).unsqueeze(0)
            video_tensor.append(frame_tensor)

    if len(video_tensor) == 0:
        return False  # No frames to process

    video_tensor = torch.cat(video_tensor, dim=0).unsqueeze(0)  # Simulate batch of size 1
    output = slowfast_model(video_tensor)

    # Simplified logic for demonstration (real applications require better output interpretation)
    behavior_detected = output.max().item() > 0.5  # Dummy threshold
    return behavior_detected

def create_collage(frames, cols=5):
    if len(frames) == 0:
        return None
    frame_width, frame_height = frames[0].size
    rows = (len(frames) + cols - 1) // cols
    collage_width = cols * frame_width
    collage_height = rows * frame_height
    collage = Image.new('RGB', (collage_width, collage_height))
    for idx, frame in enumerate(frames):
        row = idx // cols
        col = idx % cols
        collage.paste(frame, (col * frame_width, row * frame_height))
    return collage

# Streamlit app
st.title('Driver Safety Detection: Cell Phones & Head Turning')
uploaded_file = st.file_uploader('Upload a video file', type=['mp4', 'avi', 'mov'])
if uploaded_file is not None:
    video_path = f'uploaded_video.{uploaded_file.type.split("/")[1]}'
    with open(video_path, "wb") as f:
        f.write(uploaded_file.getbuffer())

    st.video(video_path)
    st.write('Running YOLOv8 inference...')
    try:
        detected_frames = process_video_with_yolo(video_path)
        st.write('YOLOv8 inference completed.')
        if detected_frames:
            st.write(f'Detected {len(detected_frames)} frames with cell phones.')
            collage_image = create_collage(detected_frames)
            if collage_image:
                st.image(collage_image, caption='Collage of detected frames')

            # Run behavior detection
            st.write('Running SlowFast behavior analysis...')
            behavior_detected = detect_behavior_with_slowfast(frames_dir='runs/detect/exp/crops/cell phone')
            if behavior_detected:
                st.write('Head turning behavior detected.')
                st.success('True Positive: Cell phone detected and head turning behavior detected.')
            else:
                st.write('No head turning behavior detected.')
                st.error('False Positive: Cell phone detected but no head turning behavior detected.')
        else:
            st.write('No cell phones detected in the video.')
            st.error('False Positive: No cell phones detected.')
    except Exception as e:
        st.write(f'Error during inference: {e}')
"""

with open("app.py", "w") as file:
    file.write(app_code)
print("app.py file has been written.")

# Start the Streamlit app in the background
import subprocess
subprocess.Popen(['streamlit', 'run', 'app.py'])

import requests
import time
from pyngrok import ngrok

# Wait a few seconds for the Streamlit app to start
time.sleep(5)

# Create a tunnel to the Streamlit app
public_url = ngrok.connect(8501)
print(f'Streamlit app running at: {public_url}')

# Expose the Streamlit app using the provided URL
!wget -q -O - https://loca.lt/mytunnelpassword


added 22 packages, and audited 23 packages in 2s

3 packages are looking for funding
  run `npm fund` for details

1 moderate severity vulnerability

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
  Cloning https://github.com/facebookresearch/fvcore to /tmp/pip-req-build-hy6npts6
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/fvcore /tmp/pip-req-build-hy6npts6
  Resolved https://github.com/facebookresearch/fvcore to commit f3b07ea37daff5b774702708134f7b702e2dc83e
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/facebookresearch/fairscale to /tmp/pip-req-build-xab_bzmo
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/fairscale /tmp/pip-req-build-xab_bzmo
  Resolved https://github.com/facebookresearch/fairscale to commit 5f484b3545f27eddb19d970fbe1d361b9c5f2b07
  Installing build dependencies ... done
  Getting require

ERROR:pyngrok.process.ngrok:t=2024-08-24T03:28:22+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2024-08-24T03:28:22+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2024-08-24T03:28:22+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

In [ ]:

!npm install -g localtunnel
# Install necessary dependencies
!pip install streamlit opencv-python-headless pyngrok
!pip install ultralytics
!pip install 'git+https://github.com/facebookresearch/fvcore'
!pip install simplejson
!pip install -U iopath
!pip install psutil
!pip install opencv-python
!pip install torchvision
!pip install tensorboard
!pip install moviepy
!pip install pytorchvideo
!pip install 'git+https://github.com/facebookresearch/fairscale'
!pip install scikit-learn

# Install PyAV and ffmpeg
!conda install av -c conda-forge

# Clone the SlowFast repository and install it
!git clone https://github.com/facebookresearch/slowfast.git
%cd slowfast

# Add SlowFast to PYTHONPATH
import os
os.environ['PYTHONPATH'] += ":/content/slowfast"

# Build and develop SlowFast
!python setup.py build develop

# Write the Streamlit app code to a file
app_code = """
import streamlit as st
import os
import shutil
import subprocess
import torch
import cv2
from PIL import Image
from ultralytics import YOLO
from slowfast.config.defaults import get_cfg
from slowfast.models import build_model
from torchvision import transforms

# YOLOv8 Model Initialization
yolo_model = YOLO('yolov8n.pt')

# SlowFast Model Initialization
cfg = get_cfg()
cfg.merge_from_file('slowfast/configs/Kinetics/SLOWFAST_8x8_R50.yaml')
cfg.NUM_GPUS = 1
cfg.TEST.CHECKPOINT_FILE_PATH = "SLOWFAST_8x8_R50.pyth"
slowfast_model = build_model(cfg)

# Load the SlowFast weights
checkpoint = torch.load(cfg.TEST.CHECKPOINT_FILE_PATH, map_location="cpu")
slowfast_model.load_state_dict(checkpoint["model_state"])
slowfast_model.eval()

def process_video_with_yolo(video_path):
    output_dir = 'runs/detect/exp'
    crops_dir = os.path.join(output_dir, 'crops', 'cell phone')
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)  # Clear previous runs

    command = ['python', '-m', 'ultralytics.run', 'detect', '--weights', 'yolov8n.pt', '--source', video_path,
               '--classes', '67', '--conf', '0.10', '--save-txt', '--save-conf', '--project', 'runs/detect', '--exist-ok', '--save-crop']
    result = subprocess.run(command, capture_output=True, text=True)

    detected_frames = []
    if os.path.exists(crops_dir):
        for file in os.listdir(crops_dir):
            if file.endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(crops_dir, file)
                img = Image.open(img_path)
                detected_frames.append(img)
    return detected_frames

def detect_behavior_with_slowfast(frames_dir):
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225]),
    ])
    video_tensor = []
    for frame_file in os.listdir(frames_dir):
        if frame_file.endswith(('.jpg', '.jpeg', '.png')):
            frame = Image.open(os.path.join(frames_dir, frame_file))
            frame_tensor = transform(frame).unsqueeze(0)
            video_tensor.append(frame_tensor)

    if len(video_tensor) == 0:
        return False  # No frames to process

    video_tensor = torch.cat(video_tensor, dim=0).unsqueeze(0)  # Simulate batch of size 1
    output = slowfast_model(video_tensor)

    # Simplified logic for demonstration (real applications require better output interpretation)
    behavior_detected = output.max().item() > 0.5  # Dummy threshold
    return behavior_detected

def create_collage(frames, cols=5):
    if len(frames) == 0:
        return None
    frame_width, frame_height = frames[0].size
    rows = (len(frames) + cols - 1) // cols
    collage_width = cols * frame_width
    collage_height = rows * frame_height
    collage = Image.new('RGB', (collage_width, collage_height))
    for idx, frame in enumerate(frames):
        row = idx // cols
        col = idx % cols
        collage.paste(frame, (col * frame_width, row * frame_height))
    return collage

# Streamlit app
st.title('Driver Safety Detection: Cell Phones & Head Turning')
uploaded_file = st.file_uploader('Upload a video file', type=['mp4', 'avi', 'mov'])
if uploaded_file is not None:
    video_path = f'uploaded_video.{uploaded_file.type.split("/")[1]}'
    with open(video_path, "wb") as f:
        f.write(uploaded_file.getbuffer())

    st.video(video_path)
    st.write('Running YOLOv8 inference...')
    try:
        detected_frames = process_video_with_yolo(video_path)
        st.write('YOLOv8 inference completed.')
        if detected_frames:
            st.write(f'Detected {len(detected_frames)} frames with cell phones.')
            collage_image = create_collage(detected_frames)
            if collage_image:
                st.image(collage_image, caption='Collage of detected frames')

            # Run behavior detection
            st.write('Running SlowFast behavior analysis...')
            behavior_detected = detect_behavior_with_slowfast(frames_dir='runs/detect/exp/crops/cell phone')
            if behavior_detected:
                st.write('Head turning behavior detected.')
                st.success('True Positive: Cell phone detected and head turning behavior detected.')
            else:
                st.write('No head turning behavior detected.')
                st.error('False Positive: Cell phone detected but no head turning behavior detected.')
        else:
            st.write('No cell phones detected in the video.')
            st.error('False Positive: No cell phones detected.')
    except Exception as e:
        st.write(f'Error during inference: {e}')
"""

with open("app.py", "w") as file:
    file.write(app_code)
print("app.py file has been written.")

# Start the Streamlit app in the background
import subprocess
subprocess.Popen(['streamlit', 'run', 'app.py'])

import requests
import time
import subprocess

# Wait a few seconds for the Streamlit app to start
time.sleep(5)

# Start LocalTunnel
localtunnel_process = subprocess.Popen(['lt', '--port', '8501'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Capture the LocalTunnel URL
public_url = None
for line in iter(localtunnel_process.stdout.readline, b''):
    decoded_line = line.decode().strip()
    print(decoded_line)
    if 'your url is:' in decoded_line:
        public_url = decoded_line.split(' ')[-1]
        break

if public_url:
    print(f'Streamlit app running at: {public_url}')
    # Make a request to the LocalTunnel URL with custom headers to bypass the password screen
    headers = {
        'User-Agent': 'Mozilla/5.0',
        'bypass-tunnel-reminder': '1'
    }
    response = requests.get(public_url, headers=headers)
    if response.status_code == 200:
        print(f'Access the Streamlit app at: {public_url}')
    else:
        print('Failed to bypass the password screen.')
else:
    print('Failed to create LocalTunnel.')

# Expose the Streamlit app using the provided URL
!wget -q -O - https://loca.lt/mytunnelpassword


changed 22 packages, and audited 23 packages in 914ms

3 packages are looking for funding
  run `npm fund` for details

1 moderate severity vulnerability

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
  Cloning https://github.com/facebookresearch/fvcore to /tmp/pip-req-build-26topit0
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/fvcore /tmp/pip-req-build-26topit0
  Resolved https://github.com/facebookresearch/fvcore to commit f3b07ea37daff5b774702708134f7b702e2dc83e
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/facebookresearch/fairscale to /tmp/pip-req-build-9x7j3y5f
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/fairscale /tmp/pip-req-build-9x7j3y5f
  Resolved https://github.com/facebookresearch/fairscale to commit 5f484b3545f27eddb19d970fbe1d361b9c5f2b07
  Installing build dependencies ... done
  Getting re

In [ ]:

!npm install -g localtunnel

# Install necessary dependencies
!pip install streamlit opencv-python-headless
!pip install ultralytics
!pip install 'git+https://github.com/facebookresearch/fvcore'
!pip install simplejson
!pip install -U iopath
!pip install psutil
!pip install opencv-python
!pip install torchvision
!pip install tensorboard
!pip install moviepy
!pip install 'git+https://github.com/facebookresearch/fairscale'
!pip install scikit-learn

# Install PyAV and ffmpeg
!conda install av -c conda-forge

# Clone the SlowFast repository and install it
!git clone https://github.com/facebookresearch/slowfast.git
%cd slowfast

# Add SlowFast to PYTHONPATH
import os
os.environ['PYTHONPATH'] += ":/content/slowfast"

# Build and develop SlowFast
!python setup.py build develop

# Fix the pytorchvideo issue by cloning and installing from the local source
%cd /content
!git clone https://github.com/facebookresearch/pytorchvideo.git
%cd pytorchvideo
!pip install -e .

# Write the Streamlit app code to a file
app_code = """
import streamlit as st
import os
import shutil
import subprocess
import torch
import cv2
from PIL import Image
from ultralytics import YOLO
from slowfast.config.defaults import get_cfg
from slowfast.models import build_model
from torchvision import transforms

# YOLOv8 Model Initialization
yolo_model = YOLO('yolov8n.pt')

# SlowFast Model Initialization
cfg = get_cfg()
cfg.merge_from_file('slowfast/configs/Kinetics/SLOWFAST_8x8_R50.yaml')
cfg.NUM_GPUS = 1
cfg.TEST.CHECKPOINT_FILE_PATH = "SLOWFAST_8x8_R50.pyth"
slowfast_model = build_model(cfg)

# Load the SlowFast weights
checkpoint = torch.load(cfg.TEST.CHECKPOINT_FILE_PATH, map_location="cpu")
slowfast_model.load_state_dict(checkpoint["model_state"])
slowfast_model.eval()

def process_video_with_yolo(video_path):
    output_dir = 'runs/detect/exp'
    crops_dir = os.path.join(output_dir, 'crops', 'cell phone')
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)  # Clear previous runs

    command = ['python', '-m', 'ultralytics.run', 'detect', '--weights', 'yolov8n.pt', '--source', video_path,
               '--classes', '67', '--conf', '0.10', '--save-txt', '--save-conf', '--project', 'runs/detect', '--exist-ok', '--save-crop']
    result = subprocess.run(command, capture_output=True, text=True)

    detected_frames = []
    if os.path.exists(crops_dir):
        for file in os.listdir(crops_dir):
            if file.endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(crops_dir, file)
                img = Image.open(img_path)
                detected_frames.append(img)
    return detected_frames

def detect_behavior_with_slowfast(frames_dir):
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225]),
    ])
    video_tensor = []
    for frame_file in os.listdir(frames_dir):
        if frame_file.endswith(('.jpg', '.jpeg', '.png')):
            frame = Image.open(os.path.join(frames_dir, frame_file))
            frame_tensor = transform(frame).unsqueeze(0)
            video_tensor.append(frame_tensor)

    if len(video_tensor) == 0:
        return False  # No frames to process

    video_tensor = torch.cat(video_tensor, dim=0).unsqueeze(0)  # Simulate batch of size 1
    output = slowfast_model(video_tensor)

    # Simplified logic for demonstration (real applications require better output interpretation)
    behavior_detected = output.max().item() > 0.5  # Dummy threshold
    return behavior_detected

def create_collage(frames, cols=5):
    if len(frames) == 0:
        return None
    frame_width, frame_height = frames[0].size
    rows = (len(frames) + cols - 1) // cols
    collage_width = cols * frame_width
    collage_height = rows * frame_height
    collage = Image.new('RGB', (collage_width, collage_height))
    for idx, frame in enumerate(frames):
        row = idx // cols
        col = idx % cols
        collage.paste(frame, (col * frame_width, row * frame_height))
    return collage

# Streamlit app
st.title('Driver Safety Detection: Cell Phones & Head Turning')
uploaded_file = st.file_uploader('Upload a video file', type=['mp4', 'avi', 'mov'])
if uploaded_file is not None:
    video_path = f'uploaded_video.{uploaded_file.type.split("/")[1]}'
    with open(video_path, "wb") as f:
        f.write(uploaded_file.getbuffer())

    st.video(video_path)
    st.write('Running YOLOv8 inference...')
    try:
        detected_frames = process_video_with_yolo(video_path)
        st.write('YOLOv8 inference completed.')
        if detected_frames:
            st.write(f'Detected {len(detected_frames)} frames with cell phones.')
            collage_image = create_collage(detected_frames)
            if collage_image:
                st.image(collage_image, caption='Collage of detected frames')

            # Run behavior detection
            st.write('Running SlowFast behavior analysis...')
            behavior_detected = detect_behavior_with_slowfast(frames_dir='runs/detect/exp/crops/cell phone')
            if behavior_detected:
                st.write('Head turning behavior detected.')
                st.success('True Positive: Cell phone detected and head turning behavior detected.')
            else:
                st.write('No head turning behavior detected.')
                st.error('False Positive: Cell phone detected but no head turning behavior detected.')
        else:
            st.write('No cell phones detected in the video.')
            st.error('False Positive: No cell phones detected.')
    except Exception as e:
        st.write(f'Error during inference: {e}')
"""

with open("app.py", "w") as file:
    file.write(app_code)
print("app.py file has been written.")


added 22 packages, and audited 23 packages in 2s

3 packages are looking for funding
  run `npm fund` for details

1 moderate severity vulnerability

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 210.6 kB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using c

In [ ]:
import requests
import time
import subprocess

# Start the Streamlit app in the background
subprocess.Popen(['streamlit', 'run', 'app.py'])

# Wait a few seconds for the Streamlit app to start
time.sleep(5)

# Start LocalTunnel
localtunnel_process = subprocess.Popen(['lt', '--port', '8501'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Capture the LocalTunnel URL
public_url = None
for line in iter(localtunnel_process.stdout.readline, b''):
    decoded_line = line.decode().strip()
    print(decoded_line)
    if 'your url is:' in decoded_line:
        public_url = decoded_line.split(' ')[-1]
        break

if public_url:
    print(f'Streamlit app running at: {public_url}')
    # Make a request to the LocalTunnel URL with custom headers to bypass the password screen
    headers = {
        'User-Agent': 'Mozilla/5.0',
        'bypass-tunnel-reminder': '1'
    }
    response = requests.get(public_url, headers=headers)
    if response.status_code == 200:
        print(f'Access the Streamlit app at: {public_url}')
    else:
        print('Failed to bypass the password screen.')
else:
    print('Failed to create LocalTunnel.')

your url is: https://tidy-knives-love.loca.lt
Streamlit app running at: https://tidy-knives-love.loca.lt
Access the Streamlit app at: https://tidy-knives-love.loca.lt


In [ ]:
!wget -q -O - https://loca.lt/mytunnelpassword

34.75.105.150

In [ ]:
# Install Node.js and npm to use LocalTunnel
!apt-get install nodejs npm

# Install LocalTunnel globally
!npm install -g localtunnel

# Install necessary Python dependencies
!pip install streamlit opencv-python-headless
!pip install ultralytics
!pip install 'git+https://github.com/facebookresearch/fvcore'
!pip install simplejson
!pip install -U iopath
!pip install psutil
!pip install opencv-python
!pip install torchvision
!pip install tensorboard
!pip install moviepy
!pip install 'git+https://github.com/facebookresearch/fairscale'
!pip install scikit-learn
!pip install pillow  # This installs PIL
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu113/torch1.9/index.html

# Install PyAV and ffmpeg
!conda install av -c conda-forge -y

# Clone the SlowFast repository and install it
!git clone https://github.com/facebookresearch/slowfast.git
%cd slowfast

# Add SlowFast to PYTHONPATH
import os
os.environ['PYTHONPATH'] += ":/content/slowfast"

# Build and develop SlowFast
!python setup.py build develop

# Fix the pytorchvideo issue by cloning and installing from the local source
%cd /content
!git clone https://github.com/facebookresearch/pytorchvideo.git
%cd pytorchvideo
!pip install -e .

# Write the Streamlit app code to a file
app_code = """
import streamlit as st
import os
import shutil
import subprocess
import torch
import cv2
from PIL import Image
from ultralytics import YOLO
from slowfast.config.defaults import get_cfg
from slowfast.models import build_model
from torchvision import transforms

# YOLOv8 Model Initialization
yolo_model = YOLO('yolov8n.pt')

# SlowFast Model Initialization
cfg = get_cfg()
cfg.merge_from_file('slowfast/configs/Kinetics/SLOWFAST_8x8_R50.yaml')
cfg.NUM_GPUS = 1
cfg.TEST.CHECKPOINT_FILE_PATH = "SLOWFAST_8x8_R50.pyth"
slowfast_model = build_model(cfg)

# Load the SlowFast weights
checkpoint = torch.load(cfg.TEST.CHECKPOINT_FILE_PATH, map_location="cpu")
slowfast_model.load_state_dict(checkpoint["model_state"])
slowfast_model.eval()

def process_video_with_yolo(video_path):
    output_dir = 'runs/detect/exp'
    crops_dir = os.path.join(output_dir, 'crops', 'cell phone')
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)  # Clear previous runs

    command = ['python', '-m', 'ultralytics.run', 'detect', '--weights', 'yolov8n.pt', '--source', video_path,
               '--classes', '67', '--conf', '0.10', '--save-txt', '--save-conf', '--project', 'runs/detect', '--exist-ok', '--save-crop']
    result = subprocess.run(command, capture_output=True, text=True)

    detected_frames = []
    if os.path.exists(crops_dir):
        for file in os.listdir(crops_dir):
            if file.endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(crops_dir, file)
                img = Image.open(img_path)
                detected_frames.append(img)
    return detected_frames

def detect_behavior_with_slowfast(frames_dir):
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225]),
    ])
    video_tensor = []
    for frame_file in os.listdir(frames_dir):
        if frame_file.endswith(('.jpg', '.jpeg', '.png')):
            frame = Image.open(os.path.join(frames_dir, frame_file))
            frame_tensor = transform(frame).unsqueeze(0)
            video_tensor.append(frame_tensor)

    if len(video_tensor) == 0:
        return False  # No frames to process

    video_tensor = torch.cat(video_tensor, dim=0).unsqueeze(0)  # Simulate batch of size 1
    output = slowfast_model(video_tensor)

    # Simplified logic for demonstration (real applications require better output interpretation)
    behavior_detected = output.max().item() > 0.5  # Dummy threshold
    return behavior_detected

def create_collage(frames, cols=5):
    if len(frames) == 0:
        return None
    frame_width, frame_height = frames[0].size
    rows = (len(frames) + cols - 1) // cols
    collage_width = cols * frame_width
    collage_height = rows * frame_height
    collage = Image.new('RGB', (collage_width, collage_height))
    for idx, frame in enumerate(frames):
        row = idx // cols
        col = idx % cols
        collage.paste(frame, (col * frame_width, row * frame_height))
    return collage

# Streamlit app
st.title('Driver Safety Detection: Cell Phones & Head Turning')
uploaded_file = st.file_uploader('Upload a video file', type=['mp4', 'avi', 'mov'])
if uploaded_file is not None:
    video_path = f'uploaded_video.{uploaded_file.type.split("/")[1]}'
    with open(video_path, "wb") as f:
        f.write(uploaded_file.getbuffer())

    st.video(video_path)
    st.write('Running YOLOv8 inference...')
    try:
        detected_frames = process_video_with_yolo(video_path)
        st.write('YOLOv8 inference completed.')
        if detected_frames:
            st.write(f'Detected {len(detected_frames)} frames with cell phones.')
            collage_image = create_collage(detected_frames)
            if collage_image:
                st.image(collage_image, caption='Collage of detected frames')

            # Run behavior detection
            st.write('Running SlowFast behavior analysis...')
            behavior_detected = detect_behavior_with_slowfast(frames_dir='runs/detect/exp/crops/cell phone')
            if behavior_detected:
                st.write('Head turning behavior detected.')
                st.success('True Positive: Cell phone detected and head turning behavior detected.')
            else:
                st.write('No head turning behavior detected.')
                st.error('False Positive: Cell phone detected but no head turning behavior detected.')
        else:
            st.write('No cell phones detected in the video.')
            st.error('False Positive: No cell phones detected.')
    except Exception as e:
        st.write(f'Error during inference: {e}')
"""

with open("app.py", "w") as file:
    file.write(app_code)
print("app.py file has been written.")

# Start the Streamlit app in the background
subprocess.Popen(['streamlit', 'run', 'app.py'])

# Wait a few seconds for the Streamlit app to start
import time
time.sleep(5)

# Start LocalTunnel
localtunnel_process = subprocess.Popen(['lt', '--port', '8501'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Capture the LocalTunnel URL
public_url = None
for line in iter(localtunnel_process.stdout.readline, b''):
    decoded_line = line.decode().strip()
    print(decoded_line)
    if 'your url is:' in decoded_line:
        public_url = decoded_line.split(' ')[-1]
        break

if public_url:
    print(f'Streamlit app running at: {public_url}')
    # Make a request to the LocalTunnel URL with custom headers to bypass the password screen
    headers = {
        'User-Agent': 'Mozilla/5.0',
        'bypass-tunnel-reminder': '1'
    }
    response = requests.get(public_url, headers=headers)
    if response.status_code == 200:
        print(f'Access the Streamlit app at: {public_url}')
    else:
        print('Failed to bypass the password screen.')
else:
    print('Failed to create LocalTunnel.')

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  gyp javascript-common libc-ares2 libjs-events libjs-highlight.js libjs-inherits
  libjs-is-typedarray libjs-psl libjs-source-map libjs-sprintf-js libjs-typedarray-to-buffer
  libnode-dev libnode72 libnotify-bin libnotify4 libuv1-dev node-abab node-abbrev node-agent-base
  node-ansi-regex node-ansi-styles node-ansistyles node-aproba node-archy node-are-we-there-yet
  node-argparse node-arrify node-asap node-asynckit node-balanced-match node-brace-expansion
  node-builtins node-cacache node-chalk node-chownr node-clean-yaml-object node-cli-table
  node-clone node-color-convert node-color-name node-colors node-columnify node-combined-stream
  node-commander node-console-control-strings node-copy-concurrently node-core-util-is
  node-coveralls node-cssom node-cssstyle node-debug node-decompress-response node-defaults
  node-delayed-stream n

NameError: name 'subprocess' is not defined

In [ ]:
!wget -q -O - https://loca.lt/mytunnelpassword

In [ ]:
# Install necessary Python dependencies
!pip install streamlit opencv-python-headless
!pip install ultralytics
!pip install 'git+https://github.com/facebookresearch/fvcore'
!pip install simplejson
!pip install -U iopath
!pip install psutil
!pip install opencv-python
!pip install torchvision
!pip install tensorboard
!pip install moviepy
!pip install 'git+https://github.com/facebookresearch/fairscale'
!pip install scikit-learn
!pip install pillow  # This installs PIL
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu113/torch1.9/index.html

# Install PyAV and ffmpeg
!conda install av -c conda-forge -y

# Install LocalTunnel globally using npm
!npm install -g localtunnel

# Clone the SlowFast repository and install it
!git clone https://github.com/facebookresearch/slowfast.git
%cd slowfast

# Add SlowFast to PYTHONPATH
import os
os.environ['PYTHONPATH'] += ":/content/slowfast"

# Build and develop SlowFast
!python setup.py build develop

# Fix the pytorchvideo issue by cloning and installing from the local source
%cd /content
!git clone https://github.com/facebookresearch/pytorchvideo.git
%cd pytorchvideo
!pip install -e .

# Write the Streamlit app code to a file
app_code = """
import streamlit as st
import os
import shutil
import subprocess
import torch
import cv2
from PIL import Image
from ultralytics import YOLO
from slowfast.config.defaults import get_cfg
from slowfast.models import build_model
from torchvision import transforms

# YOLOv8 Model Initialization
yolo_model = YOLO('yolov8n.pt')

# SlowFast Model Initialization
cfg = get_cfg()
cfg.merge_from_file('slowfast/configs/Kinetics/SLOWFAST_8x8_R50.yaml')
cfg.NUM_GPUS = 1
cfg.TEST.CHECKPOINT_FILE_PATH = "SLOWFAST_8x8_R50.pyth"
slowfast_model = build_model(cfg)

# Load the SlowFast weights
checkpoint = torch.load(cfg.TEST.CHECKPOINT_FILE_PATH, map_location="cpu")
slowfast_model.load_state_dict(checkpoint["model_state"])
slowfast_model.eval()

def process_video_with_yolo(video_path):
    output_dir = 'runs/detect/exp'
    crops_dir = os.path.join(output_dir, 'crops', 'cell phone')
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)  # Clear previous runs

    command = ['python', '-m', 'ultralytics.run', 'detect', '--weights', 'yolov8n.pt', '--source', video_path,
               '--classes', '67', '--conf', '0.10', '--save-txt', '--save-conf', '--project', 'runs/detect', '--exist-ok', '--save-crop']
    result = subprocess.run(command, capture_output=True, text=True)

    detected_frames = []
    if os.path.exists(crops_dir):
        for file in os.listdir(crops_dir):
            if file.endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(crops_dir, file)
                img = Image.open(img_path)
                detected_frames.append(img)
    return detected_frames

def detect_behavior_with_slowfast(frames_dir):
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225]),
    ])
    video_tensor = []
    for frame_file in os.listdir(frames_dir):
        if frame_file.endswith(('.jpg', '.jpeg', '.png')):
            frame = Image.open(os.path.join(frames_dir, frame_file))
            frame_tensor = transform(frame).unsqueeze(0)
            video_tensor.append(frame_tensor)

    if len(video_tensor) == 0:
        return False  # No frames to process

    video_tensor = torch.cat(video_tensor, dim=0).unsqueeze(0)  # Simulate batch of size 1
    output = slowfast_model(video_tensor)

    # Simplified logic for demonstration (real applications require better output interpretation)
    behavior_detected = output.max().item() > 0.5  # Dummy threshold
    return behavior_detected

def create_collage(frames, cols=5):
    if len(frames) == 0:
        return None
    frame_width, frame_height = frames[0].size
    rows = (len(frames) + cols - 1) // cols
    collage_width = cols * frame_width
    collage_height = rows * frame_height
    collage = Image.new('RGB', (collage_width, collage_height))
    for idx, frame in enumerate(frames):
        row = idx // cols
        col = idx % cols
        collage.paste(frame, (col * frame_width, row * frame_height))
    return collage

# Streamlit app
st.title('Driver Safety Detection: Cell Phones & Head Turning')
uploaded_file = st.file_uploader('Upload a video file', type=['mp4', 'avi', 'mov'])
if uploaded_file is not None:
    video_path = f'uploaded_video.{uploaded_file.type.split("/")[1]}'
    with open(video_path, "wb") as f:
        f.write(uploaded_file.getbuffer())

    st.video(video_path)
    st.write('Running YOLOv8 inference...')
    try:
        detected_frames = process_video_with_yolo(video_path)
        st.write('YOLOv8 inference completed.')
        if detected_frames:
            st.write(f'Detected {len(detected_frames)} frames with cell phones.')
            collage_image = create_collage(detected_frames)
            if collage_image:
                st.image(collage_image, caption='Collage of detected frames')

            # Run behavior detection
            st.write('Running SlowFast behavior analysis...')
            behavior_detected = detect_behavior_with_slowfast(frames_dir='runs/detect/exp/crops/cell phone')
            if behavior_detected:
                st.write('Head turning behavior detected.')
                st.success('True Positive: Cell phone detected and head turning behavior detected.')
            else:
                st.write('No head turning behavior detected.')
                st.error('False Positive: Cell phone detected but no head turning behavior detected.')
        else:
            st.write('No cell phones detected in the video.')
            st.error('False Positive: No cell phones detected.')
    except Exception as e:
        st.write(f'Error during inference: {e}')
"""

with open("app.py", "w") as file:
    file.write(app_code)
print("app.py file has been written.")

# Import subprocess and start the Streamlit app
import subprocess
subprocess.Popen(['streamlit', 'run', 'app.py'])

# Wait a few seconds for the Streamlit app to start
import time
time.sleep(5)

# Start LocalTunnel
localtunnel_process = subprocess.Popen(['lt', '--port', '8501'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Capture the LocalTunnel URL
public_url = None
for line in iter(localtunnel_process.stdout.readline, b''):
    decoded_line = line.decode().strip()
    print(decoded_line)
    if 'your url is:' in decoded_line:
        public_url = decoded_line.split(' ')[-1]
        break

if public_url:
    print(f'Streamlit app running at: {public_url}')
    # Make a request to the LocalTunnel URL with custom headers to bypass the password screen
    headers = {
        'User-Agent': 'Mozilla/5.0',
        'bypass-tunnel-reminder': '1'
    }
    response = requests.get(public_url, headers=headers)
    if response.status_code == 200:
        print(f'Access the Streamlit app at: {public_url}')
    else:
        print('Failed to bypass the password screen.')
else:
    print('Failed to create LocalTunnel.')

# Get the IP address for tunnel password
!wget -q -O - https://loca.lt/mytunnelpassword

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadat

NameError: name 'requests' is not defined

In [ ]:
!wget -q -O - https://loca.lt/mytunnelpassword

34.143.194.120

In [ ]:
# Install necessary Python dependencies
!pip install streamlit opencv-python-headless
!pip install ultralytics
!pip install simplejson
!pip install -U iopath
!pip install psutil
!pip install opencv-python
!pip install tensorboard
!pip install moviepy
!pip install scikit-learn
!pip install pillow  # This installs PIL

# Install PyAV and ffmpeg
!conda install av -c conda-forge -y

# Install dependencies for Detectron2
!pip install -U torch torchvision cython
!pip install -U 'git+https://github.com/facebookresearch/fvcore.git'
!pip install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

# Clone Detectron2 and install it
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo

# Install LocalTunnel globally using npm
!npm install -g localtunnel

# Clone the SlowFast repository and install it
!git clone https://github.com/facebookresearch/slowfast.git
%cd slowfast

# Add SlowFast to PYTHONPATH
import os
os.environ['PYTHONPATH'] += ":/content/slowfast"

# Build and develop SlowFast
!python setup.py build develop

# Fix the pytorchvideo issue by cloning and installing from the local source
%cd /content
!git clone https://github.com/facebookresearch/pytorchvideo.git
%cd pytorchvideo
!pip install -e .

# Write the Streamlit app code to a file
app_code = """
import streamlit as st
import os
import shutil
import subprocess
import torch
import cv2
from PIL import Image
from ultralytics import YOLO
from slowfast.config.defaults import get_cfg
from slowfast.models import build_model
from torchvision import transforms

# YOLOv8 Model Initialization
yolo_model = YOLO('yolov8n.pt')

# SlowFast Model Initialization
cfg = get_cfg()
cfg.merge_from_file('slowfast/configs/Kinetics/SLOWFAST_8x8_R50.yaml')
cfg.NUM_GPUS = 1
cfg.TEST.CHECKPOINT_FILE_PATH = "SLOWFAST_8x8_R50.pyth"
slowfast_model = build_model(cfg)

# Load the SlowFast weights
checkpoint = torch.load(cfg.TEST.CHECKPOINT_FILE_PATH, map_location="cpu")
slowfast_model.load_state_dict(checkpoint["model_state"])
slowfast_model.eval()

def process_video_with_yolo(video_path):
    output_dir = 'runs/detect/exp'
    crops_dir = os.path.join(output_dir, 'crops', 'cell phone')
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)  # Clear previous runs

    command = ['python', '-m', 'ultralytics.run', 'detect', '--weights', 'yolov8n.pt', '--source', video_path,
               '--classes', '67', '--conf', '0.10', '--save-txt', '--save-conf', '--project', 'runs/detect', '--exist-ok', '--save-crop']
    result = subprocess.run(command, capture_output=True, text=True)

    detected_frames = []
    if os.path.exists(crops_dir):
        for file in os.listdir(crops_dir):
            if file.endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(crops_dir, file)
                img = Image.open(img_path)
                detected_frames.append(img)
    return detected_frames

def detect_behavior_with_slowfast(frames_dir):
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225]),
    ])
    video_tensor = []
    for frame_file in os.listdir(frames_dir):
        if frame_file.endswith(('.jpg', '.jpeg', '.png')):
            frame = Image.open(os.path.join(frames_dir, frame_file))
            frame_tensor = transform(frame).unsqueeze(0)
            video_tensor.append(frame_tensor)

    if len(video_tensor) == 0:
        return False  # No frames to process

    video_tensor = torch.cat(video_tensor, dim=0).unsqueeze(0)  # Simulate batch of size 1
    output = slowfast_model(video_tensor)

    # Simplified logic for demonstration (real applications require better output interpretation)
    behavior_detected = output.max().item() > 0.5  # Dummy threshold
    return behavior_detected

def create_collage(frames, cols=5):
    if len(frames) == 0:
        return None
    frame_width, frame_height = frames[0].size
    rows = (len(frames) + cols - 1) // cols
    collage_width = cols * frame_width
    collage_height = rows * frame_height
    collage = Image.new('RGB', (collage_width, collage_height))
    for idx, frame in enumerate(frames):
        row = idx // cols
        col = idx % cols
        collage.paste(frame, (col * frame_width, row * frame_height))
    return collage

# Streamlit app
st.title('Driver Safety Detection: Cell Phones & Head Turning')
uploaded_file = st.file_uploader('Upload a video file', type=['mp4', 'avi', 'mov'])
if uploaded_file is not None:
    video_path = f'uploaded_video.{uploaded_file.type.split("/")[1]}'
    with open(video_path, "wb") as f:
        f.write(uploaded_file.getbuffer())

    st.video(video_path)
    st.write('Running YOLOv8 inference...')
    try:
        detected_frames = process_video_with_yolo(video_path)
        st.write('YOLOv8 inference completed.')
        if detected_frames:
            st.write(f'Detected {len(detected_frames)} frames with cell phones.')
            collage_image = create_collage(detected_frames)
            if collage_image:
                st.image(collage_image, caption='Collage of detected frames')

            # Run behavior detection
            st.write('Running SlowFast behavior analysis...')
            behavior_detected = detect_behavior_with_slowfast(frames_dir='runs/detect/exp/crops/cell phone')
            if behavior_detected:
                st.write('Head turning behavior detected.')
                st.success('True Positive: Cell phone detected and head turning behavior detected.')
            else:
                st.write('No head turning behavior detected.')
                st.error('False Positive: Cell phone detected but no head turning behavior detected.')
        else:
            st.write('No cell phones detected in the video.')
            st.error('False Positive: No cell phones detected.')
    except Exception as e:
        st.write(f'Error during inference: {e}')
"""

with open("app.py", "w") as file:
    file.write(app_code)
print("app.py file has been written.")

# Import subprocess and start the Streamlit app
import subprocess
subprocess.Popen(['streamlit', 'run', 'app.py'])

# Wait a few seconds for the Streamlit app to start
import time
time.sleep(5)

# Start LocalTunnel
localtunnel_process = subprocess.Popen(['lt', '--port', '8501'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Capture the LocalTunnel URL
public_url = None
for line in iter(localtunnel_process.stdout.readline, b''):
    decoded_line = line.decode().strip()
    print(decoded_line)
    if 'your url is:' in decoded_line:
        public_url = decoded_line.split(' ')[-1]
        break

if public_url:
    print(f'Streamlit app running at: {public_url}')
    # Make a request to the LocalTunnel URL with custom headers to bypass the password screen
    headers = {
        'User-Agent': 'Mozilla/5.0',
        'bypass-tunnel-reminder': '1'
    }
    response = requests.get(public_url, headers=headers)
    if response.status_code == 200:
        print(f'Access the Streamlit app at: {public_url}')
    else:
        print('Failed to bypass the password screen.')
else:
    print('Failed to create LocalTunnel.')

# Get the IP address for tunnel password
!wget -q -O - https://loca.lt/mytunnelpassword

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 822.5 kB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.meta


added 22 packages, and audited 23 packages in 2s

3 packages are looking for funding
  run `npm fund` for details

1 moderate severity vulnerability

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
Cloning into 'slowfast'...
remote: Enumerating objects: 2149, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 2149 (delta 46), reused 59 (delta 35), pack-reused 2013 (from 1)
Receiving objects: 100% (2149/2149), 31.51 MiB | 40.53 MiB/s, done.
Resolving deltas: 100% (1457/1457), done.
/content/slowfast
running build
running build_py
creating build
creating build/lib
creating build/lib/slowfast
copying slowfast/__init__.py -> build/lib/slowfast
creating build/lib/slowfast/models
copying slowfast/models/stem_helper.py -> build/lib/slowfast/models
copying slowfast/models/optimizer.py -> build/lib/slowfast/models
copying slowfast/models/reversible_mvit.py -> build

NameError: name 'requests' is not defined

In [ ]:
!wget -q -O - https://loca.lt/mytunnelpassword

34.19.44.211

In [ ]:
# Install necessary Python dependencies
!pip install streamlit opencv-python-headless
!pip install ultralytics
!pip install simplejson
!pip install -U iopath
!pip install psutil
!pip install opencv-python
!pip install tensorboard
!pip install moviepy
!pip install scikit-learn
!pip install pillow  # This installs PIL

# Install PyAV and ffmpeg
!conda install av -c conda-forge -y

# Install dependencies for Detectron2
!pip install -U torch torchvision cython
!pip install -U 'git+https://github.com/facebookresearch/fvcore.git'
!pip install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

# Clone Detectron2 and install it
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo

# Install LocalTunnel globally using npm
!npm install -g localtunnel

# Clone the SlowFast repository and install it
!git clone https://github.com/facebookresearch/slowfast.git
%cd slowfast

# Add SlowFast to PYTHONPATH
import os
os.environ['PYTHONPATH'] += ":/content/slowfast"

# Build and develop SlowFast
!python setup.py build develop

# Fix the pytorchvideo issue by cloning and installing from the local source
%cd /content
!git clone https://github.com/facebookresearch/pytorchvideo.git
%cd pytorchvideo
!pip install -e .

# Write the Streamlit app code to a file
app_code = """
import streamlit as st
import os
import shutil
import subprocess
import torch
import cv2
from PIL import Image
from ultralytics import YOLO
from slowfast.config.defaults import get_cfg
from slowfast.models import build_model
from torchvision import transforms

# YOLOv8 Model Initialization
yolo_model = YOLO('yolov8n.pt')

# SlowFast Model Initialization
cfg = get_cfg()
cfg.merge_from_file('/content/slowfast/configs/Kinetics/SLOWFAST_8x8_R50.yaml')
cfg.NUM_GPUS = 0
cfg.TEST.CHECKPOINT_FILE_PATH = "SLOWFAST_8x8_R50.pyth"
slowfast_model = build_model(cfg)

# Load the SlowFast weights
checkpoint = torch.load(cfg.TEST.CHECKPOINT_FILE_PATH, map_location="cpu")
slowfast_model.load_state_dict(checkpoint["model_state"])
slowfast_model.eval()

def process_video_with_yolo(video_path):
    output_dir = 'runs/detect/exp'
    crops_dir = os.path.join(output_dir, 'crops', 'cell phone')
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)  # Clear previous runs

    command = ['python', '-m', 'ultralytics.run', 'detect', '--weights', 'yolov8n.pt', '--source', video_path,
               '--classes', '67', '--conf', '0.10', '--save-txt', '--save-conf', '--project', 'runs/detect', '--exist-ok', '--save-crop']
    result = subprocess.run(command, capture_output=True, text=True)

    detected_frames = []
    if os.path.exists(crops_dir):
        for file in os.listdir(crops_dir):
            if file.endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(crops_dir, file)
                img = Image.open(img_path)
                detected_frames.append(img)
    return detected_frames

def detect_behavior_with_slowfast(frames_dir):
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225]),
    ])
    video_tensor = []
    for frame_file in os.listdir(frames_dir):
        if frame_file.endswith(('.jpg', '.jpeg', '.png')):
            frame = Image.open(os.path.join(frames_dir, frame_file))
            frame_tensor = transform(frame).unsqueeze(0)
            video_tensor.append(frame_tensor)

    if len(video_tensor) == 0:
        return False  # No frames to process

    video_tensor = torch.cat(video_tensor, dim=0).unsqueeze(0)  # Simulate batch of size 1
    output = slowfast_model(video_tensor)

    # Simplified logic for demonstration (real applications require better output interpretation)
    behavior_detected = output.max().item() > 0.5  # Dummy threshold
    return behavior_detected

def create_collage(frames, cols=5):
    if len(frames) == 0:
        return None
    frame_width, frame_height = frames[0].size
    rows = (len(frames) + cols - 1) // cols
    collage_width = cols * frame_width
    collage_height = rows * frame_height
    collage = Image.new('RGB', (collage_width, collage_height))
    for idx, frame in enumerate(frames):
        row = idx // cols
        col = idx % cols
        collage.paste(frame, (col * frame_width, row * frame_height))
    return collage

# Streamlit app
st.title('Driver Safety Detection: Cell Phones & Head Turning')
uploaded_file = st.file_uploader('Upload a video file', type=['mp4', 'avi', 'mov'])
if uploaded_file is not None:
    video_path = f'uploaded_video.{uploaded_file.type.split("/")[1]}'
    with open(video_path, "wb") as f:
        f.write(uploaded_file.getbuffer())

    st.video(video_path)
    st.write('Running YOLOv8 inference...')
    try:
        detected_frames = process_video_with_yolo(video_path)
        st.write('YOLOv8 inference completed.')
        if detected_frames:
            st.write(f'Detected {len(detected_frames)} frames with cell phones.')
            collage_image = create_collage(detected_frames)
            if collage_image:
                st.image(collage_image, caption='Collage of detected frames')

            # Run behavior detection
            st.write('Running SlowFast behavior analysis...')
            behavior_detected = detect_behavior_with_slowfast(frames_dir='runs/detect/exp/crops/cell phone')
            if behavior_detected:
                st.write('Head turning behavior detected.')
                st.success('True Positive: Cell phone detected and head turning behavior detected.')
            else:
                st.write('No head turning behavior detected.')
                st.error('False Positive: Cell phone detected but no head turning behavior detected.')
        else:
            st.write('No cell phones detected in the video.')
            st.error('False Positive: No cell phones detected.')
    except Exception as e:
        st.write(f'Error during inference: {e}')
"""

with open("app.py", "w") as file:
    file.write(app_code)
print("app.py file has been written.")

# Import subprocess and start the Streamlit app
import subprocess
subprocess.Popen(['streamlit', 'run', 'app.py'])

# Wait a few seconds for the Streamlit app to start
import time
time.sleep(5)

# Start LocalTunnel
localtunnel_process = subprocess.Popen(['lt', '--port', '8501'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Capture the LocalTunnel URL
public_url = None
for line in iter(localtunnel_process.stdout.readline, b''):
    decoded_line = line.decode().strip()
    print(decoded_line)
    if 'your url is:' in decoded_line:
        public_url = decoded_line.split(' ')[-1]
        break

if public_url:
    print(f'Streamlit app running at: {public_url}')
    # Make a request to the LocalTunnel URL with custom headers to bypass the password screen
    headers = {
        'User-Agent': 'Mozilla/5.0',
        'bypass-tunnel-reminder': '1'
    }
    response = requests.get(public_url, headers=headers)
    if response.status_code == 200:
        print(f'Access the Streamlit app at: {public_url}')
    else:
        print('Failed to bypass the password screen.')
else:
    print('Failed to create LocalTunnel.')

# Get the IP address for tunnel password
!wget -q -O - https://loca.lt/mytunnelpassword

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.1/871.1 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31529 sha256=8


added 22 packages, and audited 23 packages in 2s

3 packages are looking for funding
  run `npm fund` for details

1 moderate severity vulnerability

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
Cloning into 'slowfast'...
remote: Enumerating objects: 2149, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 2149 (delta 46), reused 59 (delta 35), pack-reused 2013 (from 1)
Receiving objects: 100% (2149/2149), 31.51 MiB | 25.59 MiB/s, done.
Resolving deltas: 100% (1457/1457), done.
/content/slowfast
running build
running build_py
creating build
creating build/lib
creating build/lib/slowfast
copying slowfast/__init__.py -> build/lib/slowfast
creating build/lib/slowfast/models
copying slowfast/models/masked.py -> build/lib/slowfast/models
copying slowfast/models/common.py -> build/lib/slowfast/models
copying slowfast/models/__init__.py -> build/lib/slowfast/m

NameError: name 'requests' is not defined

In [ ]:
!wget -q -O - https://loca.lt/mytunnelpassword

34.125.186.177

In [ ]:
import os
import torch
from slowfast.config.defaults import get_cfg
from slowfast.models import build_model

# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# SlowFast Model Initialization
cfg = get_cfg()
cfg.merge_from_file('/content/slowfast/configs/Kinetics/SLOWFAST_8x8_R50.yaml')

# Set NUM_GPUs based on actual GPU availability
cfg.NUM_GPUS = 1 if torch.cuda.is_available() else 0

# Check if the .pyth file is in the current directory
if os.path.exists("SLOWFAST_8x8_R50.pyth"):
    cfg.TEST.CHECKPOINT_FILE_PATH = "SLOWFAST_8x8_R50.pyth"
elif os.path.exists("/content/SLOWFAST_8x8_R50.pyth"):
    cfg.TEST.CHECKPOINT_FILE_PATH = "/content/SLOWFAST_8x8_R50.pyth"
else:
    raise FileNotFoundError("SLOWFAST_8x8_R50.pyth not found in the current directory or in /content/")

# Build the model
slowfast_model = build_model(cfg)

# Load the model weights
try:
    checkpoint = torch.load(cfg.TEST.CHECKPOINT_FILE_PATH, map_location=device)
    slowfast_model.load_state_dict(checkpoint["model_state"])
    slowfast_model.to(device)  # Move model to GPU if available
    slowfast_model.eval()
    print("Model loaded successfully")
except Exception as e:
    print(f"Error loading model: {e}")
    raise

# Print model's current device
print(f"Model is on device: {next(slowfast_model.parameters()).device}")

ModuleNotFoundError: No module named 'slowfast.config'

In [ ]:
import os
import sys
import torch

# Ensure SlowFast is in the Python path
slowfast_path = '/content/slowfast'
if slowfast_path not in sys.path:
    sys.path.append(slowfast_path)

# Now we can import from slowfast
from slowfast.config.defaults import get_cfg
from slowfast.models import build_model

# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# SlowFast Model Initialization
cfg = get_cfg()
cfg.merge_from_file('/content/slowfast/configs/Kinetics/SLOWFAST_8x8_R50.yaml')

# Set NUM_GPUs based on actual GPU availability
cfg.NUM_GPUS = 1 if torch.cuda.is_available() else 0

# Check if the .pyth file is in the current directory
if os.path.exists("SLOWFAST_8x8_R50.pyth"):
    cfg.TEST.CHECKPOINT_FILE_PATH = "SLOWFAST_8x8_R50.pyth"
elif os.path.exists("/content/SLOWFAST_8x8_R50.pyth"):
    cfg.TEST.CHECKPOINT_FILE_PATH = "/content/SLOWFAST_8x8_R50.pyth"
else:
    raise FileNotFoundError("SLOWFAST_8x8_R50.pyth not found in the current directory or in /content/")

# Build the model
slowfast_model = build_model(cfg)

# Load the model weights
try:
    checkpoint = torch.load(cfg.TEST.CHECKPOINT_FILE_PATH, map_location=device)
    slowfast_model.load_state_dict(checkpoint["model_state"])
    slowfast_model.to(device)  # Move model to GPU if available
    slowfast_model.eval()
    print("Model loaded successfully")
except Exception as e:
    print(f"Error loading model: {e}")
    raise

# Print model's current device
print(f"Model is on device: {next(slowfast_model.parameters()).device}")

ModuleNotFoundError: No module named 'slowfast.config'

In [ ]:
!ls /content
print("\nContents of the SlowFast directory:")
!ls /content/slowfast

detectron2_repo  pytorchvideo  sample_data  slowfast  SLOWFAST_8x8_R50.pyth

Contents of the SlowFast directory:
ava_evaluation	    CONTRIBUTING.md	LICENSE       README.md  slowfast.egg-info
build		    demo		linter.sh     setup.cfg  tools
CODE_OF_CONDUCT.md  GETTING_STARTED.md	MODEL_ZOO.md  setup.py	 VISUALIZATION_TOOLS.md
configs		    INSTALL.md		projects      slowfast


In [ ]:
%cd /content/slowfast
!pip install -e .
%cd /content

/content/slowfast
Obtaining file:///content/slowfast
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of slowfast to determine which version is compatible with other requirements. This could take a while.
ERROR: Could not find a version that satisfies the requirement PIL (from slowfast) (from versions: none)
ERROR: No matching distribution found for PIL
/content


In [ ]:
import sys
print("Python path:")
print('\n'.join(sys.path))

print("\nTrying to import slowfast:")
try:
    import slowfast
    print("SlowFast imported successfully")
    print(f"SlowFast path: {slowfast.__file__}")
except ImportError as e:
    print(f"Import failed: {e}")

Python path:
/content
/env/python
/usr/lib/python310.zip
/usr/lib/python3.10
/usr/lib/python3.10/lib-dynload

/usr/local/lib/python3.10/dist-packages
/usr/lib/python3/dist-packages
/usr/local/lib/python3.10/dist-packages/IPython/extensions
/usr/local/lib/python3.10/dist-packages/setuptools/_vendor
/root/.ipython
/content/slowfast

Trying to import slowfast:
SlowFast imported successfully
SlowFast path: None


In [ ]:
import sys
sys.path.insert(0, '/content/slowfast')

print("Trying to import slowfast:")
try:
    import slowfast
    print("SlowFast imported successfully")
    print(f"SlowFast path: {slowfast.__file__}")

    from slowfast.config.defaults import get_cfg
    print("get_cfg imported successfully")
except ImportError as e:
    print(f"Import failed: {e}")

print("\nPython path:")
print('\n'.join(sys.path))

Trying to import slowfast:
SlowFast imported successfully
SlowFast path: None
Import failed: No module named 'slowfast.config'

Python path:
/content/slowfast
/content/slowfast
/content/slowfast
/content
/env/python
/usr/lib/python310.zip
/usr/lib/python3.10
/usr/lib/python3.10/lib-dynload

/usr/local/lib/python3.10/dist-packages
/usr/lib/python3/dist-packages
/usr/local/lib/python3.10/dist-packages/IPython/extensions
/usr/local/lib/python3.10/dist-packages/setuptools/_vendor
/root/.ipython
/content/slowfast


In [ ]:
!ls /content/slowfast
print("\nContents of /content/slowfast/slowfast:")
!ls /content/slowfast/slowfast
print("\nContents of /content/slowfast/slowfast/config:")
!ls /content/slowfast/slowfast/config

ava_evaluation	    CONTRIBUTING.md	LICENSE       README.md  slowfast.egg-info
build		    demo		linter.sh     setup.cfg  tools
CODE_OF_CONDUCT.md  GETTING_STARTED.md	MODEL_ZOO.md  setup.py	 VISUALIZATION_TOOLS.md
configs		    INSTALL.md		projects      slowfast

Contents of /content/slowfast/slowfast:
config	datasets  __init__.py  models  __pycache__  utils  visualization

Contents of /content/slowfast/slowfast/config:
custom_config.py  defaults.py  __init__.py  __pycache__


In [ ]:
import sys
import os

# Add the parent directory of the slowfast package to the Python path
slowfast_path = '/content/slowfast'
if slowfast_path not in sys.path:
    sys.path.insert(0, slowfast_path)

print("Updated Python path:")
print('\n'.join(sys.path))

print("\nTrying to import slowfast:")
try:
    from slowfast.config.defaults import get_cfg
    print("Successfully imported get_cfg from slowfast.config.defaults")

    import slowfast
    print(f"SlowFast package location: {slowfast.__file__}")
except ImportError as e:
    print(f"Import failed: {e}")

Updated Python path:
/content/slowfast
/content/slowfast
/content/slowfast
/content
/env/python
/usr/lib/python310.zip
/usr/lib/python3.10
/usr/lib/python3.10/lib-dynload

/usr/local/lib/python3.10/dist-packages
/usr/lib/python3/dist-packages
/usr/local/lib/python3.10/dist-packages/IPython/extensions
/usr/local/lib/python3.10/dist-packages/setuptools/_vendor
/root/.ipython
/content/slowfast

Trying to import slowfast:
Import failed: No module named 'slowfast.config'


In [ ]:
# Install necessary Python dependencies
!pip install streamlit opencv-python-headless
!pip install ultralytics
!pip install simplejson
!pip install -U iopath
!pip install psutil
!pip install opencv-python
!pip install tensorboard
!pip install moviepy
!pip install scikit-learn
!pip install pillow  # This installs PIL
!pip install pytorchvideo
!pip install 'git+https://github.com/facebookresearch/fairscale.git'

# Install PyAV and ffmpeg
!conda install av -c conda-forge -y
!conda install -c conda-forge ffmpeg -y

# Install dependencies for Detectron2
!pip install -U torch torchvision cython
!pip install -U 'git+https://github.com/facebookresearch/fvcore.git'
!pip install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

# Clone Detectron2 and install it
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo

# Install LocalTunnel globally using npm
!npm install -g localtunnel

# Clone the SlowFast repository and install it
!git clone https://github.com/facebookresearch/slowfast.git
%cd slowfast

# Add SlowFast to PYTHONPATH
import os
os.environ['PYTHONPATH'] += ":/content/slowfast"

# Build and develop SlowFast
!python setup.py build develop

# Ensure Python >= 3.8
!python --version

# Fix the pytorchvideo issue by cloning and installing from the local source
%cd /content
!git clone https://github.com/facebookresearch/pytorchvideo.git
%cd pytorchvideo
!pip install -e .

# Write the Streamlit app code to a file
app_code = """
import streamlit as st
import os
import shutil
import subprocess
import torch
import cv2
from PIL import Image
from ultralytics import YOLO
from torchvision import transforms

# YOLOv8 Model Initialization
yolo_model = YOLO('yolov8n.pt')

# SlowFast Model Initialization using torch.hub
slowfast_model = torch.hub.load('facebookresearch/pytorchvideo', 'slowfast_r50', pretrained=True)
slowfast_model.eval()

def process_video_with_yolo(video_path):
    output_dir = 'runs/detect/exp'
    crops_dir = os.path.join(output_dir, 'crops', 'cell phone')
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)  # Clear previous runs

    command = ['python', '-m', 'ultralytics.run', 'detect', '--weights', 'yolov8n.pt', '--source', video_path,
               '--classes', '67', '--conf', '0.10', '--save-txt', '--save-conf', '--project', 'runs/detect', '--exist-ok', '--save-crop']
    result = subprocess.run(command, capture_output=True, text=True)

    detected_frames = []
    if os.path.exists(crops_dir):
        for file in os.listdir(crops_dir):
            if file.endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(crops_dir, file)
                img = Image.open(img_path)
                detected_frames.append(img)
    return detected_frames

def detect_behavior_with_slowfast(frames):
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225]),
    ])

    video_tensor = torch.stack([transform(frame) for frame in frames]).unsqueeze(0)

    # Ensure we have at least 32 frames (pad if necessary)
    if video_tensor.size(1) < 32:
        pad_size = 32 - video_tensor.size(1)
        video_tensor = torch.nn.functional.pad(video_tensor, (0, 0, 0, 0, 0, pad_size))

    # SlowFast models expect input in the format [B, C, T, H, W]
    video_tensor = video_tensor.permute(0, 2, 1, 3, 4)

    with torch.no_grad():
        output = slowfast_model(video_tensor)

    # The output is a dictionary, we're interested in the 'logits'
    logits = output['logits']

    # Get the predicted class (you may want to adjust this logic based on your specific use case)
    predicted_class = torch.argmax(logits, dim=1)

    # For demonstration, let's say behavior is detected if the predicted class is above a certain threshold
    behavior_detected = predicted_class.item() > 50  # Adjust this threshold as needed

    return behavior_detected

def create_collage(frames, cols=5):
    if len(frames) == 0:
        return None
    frame_width, frame_height = frames[0].size
    rows = (len(frames) + cols - 1) // cols
    collage_width = cols * frame_width
    collage_height = rows * frame_height
    collage = Image.new('RGB', (collage_width, collage_height))
    for idx, frame in enumerate(frames):
        row = idx // cols
        col = idx % cols
        collage.paste(frame, (col * frame_width, row * frame_height))
    return collage

# Streamlit app
st.title('Driver Safety Detection: Cell Phones & Head Turning')
uploaded_file = st.file_uploader('Upload a video file', type=['mp4', 'avi', 'mov'])
if uploaded_file is not None:
    video_path = f'uploaded_video.{uploaded_file.type.split("/")[1]}'
    with open(video_path, "wb") as f:
        f.write(uploaded_file.getbuffer())

    st.video(video_path)
    st.write('Running YOLOv8 inference...')
    try:
        detected_frames = process_video_with_yolo(video_path)
        st.write('YOLOv8 inference completed.')
        if detected_frames:
            st.write(f'Detected {len(detected_frames)} frames with cell phones.')
            collage_image = create_collage(detected_frames)
            if collage_image:
                st.image(collage_image, caption='Collage of detected frames')

            # Run behavior detection
            st.write('Running SlowFast behavior analysis...')
            behavior_detected = detect_behavior_with_slowfast(detected_frames)
            if behavior_detected:
                st.write('Head turning behavior detected.')
                st.success('True Positive: Cell phone detected and head turning behavior detected.')
            else:
                st.write('No head turning behavior detected.')
                st.error('False Positive: Cell phone detected but no head turning behavior detected.')
        else:
            st.write('No cell phones detected in the video.')
            st.error('False Positive: No cell phones detected.')
    except Exception as e:
        st.write(f'Error during inference: {e}')
"""

with open("app.py", "w") as file:
    file.write(app_code)
print("app.py file has been written.")

# Import subprocess and start the Streamlit app
import subprocess
subprocess.Popen(['streamlit', 'run', 'app.py'])

# Wait a few seconds for the Streamlit app to start
import time
time.sleep(5)

# Start LocalTunnel
localtunnel_process = subprocess.Popen(['lt', '--port', '8501'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Capture the LocalTunnel URL
public_url = None
for line in iter(localtunnel_process.stdout.readline, b''):
    decoded_line = line.decode().strip()
    print(decoded_line)
    if 'your url is:' in decoded_line:
        public_url = decoded_line.split(' ')[-1]
        break

if public_url:
    print(f'Streamlit app running at: {public_url}')
    # Make a request to the LocalTunnel URL with custom headers to bypass the password screen
    headers = {
        'User-Agent': 'Mozilla/5.0',
        'bypass-tunnel-reminder': '1'
    }
    response = requests.get(public_url, headers=headers)
    if response.status_code == 200:
        print(f'Access the Streamlit app at: {public_url}')
    else:
        print('Failed to bypass the password screen.')
else:
    print('Failed to create LocalTunnel.')

# Get the IP address for tunnel password
!wget -q -O - https://loca.lt/mytunnelpassword

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 108.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.1/871.1 kB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31529 sha256


added 22 packages, and audited 23 packages in 2s

3 packages are looking for funding
  run `npm fund` for details

1 moderate severity vulnerability

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
Cloning into 'slowfast'...
remote: Enumerating objects: 2149, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (112/112), done.
remote: Total 2149 (delta 45), reused 70 (delta 24), pack-reused 2013 (from 1)
Receiving objects: 100% (2149/2149), 31.50 MiB | 11.71 MiB/s, done.
Resolving deltas: 100% (1455/1455), done.
/content/slowfast
running build
running build_py
creating build
creating build/lib
creating build/lib/slowfast
copying slowfast/__init__.py -> build/lib/slowfast
creating build/lib/slowfast/models
copying slowfast/models/masked.py -> build/lib/slowfast/models
copying slowfast/models/common.py -> build/lib/slowfast/models
copying slowfast/models/__init__.py -> build/lib/slowfast/m

NameError: name 'requests' is not defined

In [ ]:
!wget -q -O - https://loca.lt/mytunnelpassword

34.124.254.195

In [ ]:
# Install necessary Python dependencies
!pip install streamlit opencv-python-headless
!pip install ultralytics
!pip install simplejson
!pip install -U iopath
!pip install psutil
!pip install opencv-python
!pip install tensorboard
!pip install moviepy
!pip install scikit-learn
!pip install pillow  # This installs PIL
!pip install pytorchvideo
!pip install 'git+https://github.com/facebookresearch/fairscale.git'

# Install PyAV and ffmpeg
!conda install av -c conda-forge -y
!conda install -c conda-forge ffmpeg -y

# Install dependencies for Detectron2
!pip install -U torch torchvision cython
!pip install -U 'git+https://github.com/facebookresearch/fvcore.git'
!pip install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

# Clone Detectron2 and install it
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo

# Install LocalTunnel globally using npm
!npm install -g localtunnel

# Clone the SlowFast repository and install it
!git clone https://github.com/facebookresearch/slowfast.git
%cd slowfast

# Add SlowFast to PYTHONPATH
import os
os.environ['PYTHONPATH'] += ":/content/slowfast"

# Build and develop SlowFast
!python setup.py build develop

# Ensure Python >= 3.8
!python --version

# Fix the pytorchvideo issue by cloning and installing from the local source
%cd /content
!git clone https://github.com/facebookresearch/pytorchvideo.git
%cd pytorchvideo
!pip install -e .

# Write the Streamlit app code to a file
app_code = """
import streamlit as st
import os
import shutil
import subprocess
import torch
import cv2
from PIL import Image
from ultralytics import YOLO
from torchvision import transforms

# YOLOv8 Model Initialization
yolo_model = YOLO('yolov8n.pt')

# SlowFast Model Initialization using torch.hub
slowfast_model = torch.hub.load('facebookresearch/pytorchvideo', 'slowfast_r50', pretrained=True)
slowfast_model.eval()

def process_video_with_yolo(video_path):
    # Load video frames
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()

    # Run inference on video frames
    results = yolo_model(frames, save=True, save_txt=True, conf=0.10, classes=67)
    detected_frames = []

    for result in results:
        crops = result.crop(save=True)  # Save cropped images containing detected objects
        for crop in crops:
            img = Image.fromarray(crop["im"])
            detected_frames.append(img)

    return detected_frames

def detect_behavior_with_slowfast(frames):
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225]),
    ])

    video_tensor = torch.stack([transform(frame) for frame in frames]).unsqueeze(0)

    # Ensure we have at least 32 frames (pad if necessary)
    if video_tensor.size(1) < 32:
        pad_size = 32 - video_tensor.size(1)
        video_tensor = torch.nn.functional.pad(video_tensor, (0, 0, 0, 0, 0, pad_size))

    # SlowFast models expect input in the format [B, C, T, H, W]
    video_tensor = video_tensor.permute(0, 2, 1, 3, 4)

    with torch.no_grad():
        output = slowfast_model(video_tensor)

    # The output is a dictionary, we're interested in the 'logits'
    logits = output['logits']

    # Get the predicted class (you may want to adjust this logic based on your specific use case)
    predicted_class = torch.argmax(logits, dim=1)

    # For demonstration, let's say behavior is detected if the predicted class is above a certain threshold
    behavior_detected = predicted_class.item() > 50  # Adjust this threshold as needed

    return behavior_detected

def create_collage(frames, cols=5):
    if len(frames) == 0:
        return None
    frame_width, frame_height = frames[0].size
    rows = (len(frames) + cols - 1) // cols
    collage_width = cols * frame_width
    collage_height = rows * frame_height
    collage = Image.new('RGB', (collage_width, collage_height))
    for idx, frame in enumerate(frames):
        row = idx // cols
        col = idx % cols
        collage.paste(frame, (col * frame_width, row * frame_height))
    return collage

# Streamlit app
st.title('Driver Safety Detection: Cell Phones & Head Turning')
uploaded_file = st.file_uploader('Upload a video file', type=['mp4', 'avi', 'mov'])
if uploaded_file is not None:
    video_path = f'uploaded_video.{uploaded_file.type.split("/")[1]}'
    with open(video_path, "wb") as f:
        f.write(uploaded_file.getbuffer())

    st.video(video_path)
    st.write('Running YOLOv8 inference...')
    try:
        detected_frames = process_video_with_yolo(video_path)
        st.write('YOLOv8 inference completed.')
        if detected_frames:
            st.write(f'Detected {len(detected_frames)} frames with cell phones.')
            collage_image = create_collage(detected_frames)
            if collage_image:
                st.image(collage_image, caption='Collage of detected frames')

            # Run behavior detection
            st.write('Running SlowFast behavior analysis...')
            behavior_detected = detect_behavior_with_slowfast(detected_frames)
            if behavior_detected:
                st.write('Head turning behavior detected.')
                st.success('True Positive: Cell phone detected and head turning behavior detected.')
            else:
                st.write('No head turning behavior detected.')
                st.error('False Positive: Cell phone detected but no head turning behavior detected.')
        else:
            st.write('No cell phones detected in the video.')
            st.error('False Positive: No cell phones detected.')
    except Exception as e:
        st.write(f'Error during inference: {e}')
"""

with open("app.py", "w") as file:
    file.write(app_code)
print("app.py file has been written.")

# Import subprocess and start the Streamlit app
import subprocess
subprocess.Popen(['streamlit', 'run', 'app.py'])

# Wait a few seconds for the Streamlit app to start
import time
time.sleep(5)

# Start LocalTunnel
localtunnel_process = subprocess.Popen(['lt', '--port', '8501'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Capture the LocalTunnel URL
public_url = None
for line in iter(localtunnel_process.stdout.readline, b''):
    decoded_line = line.decode().strip()
    print(decoded_line)
    if 'your url is:' in decoded_line:
        public_url = decoded_line.split(' ')[-1]
        break

if public_url:
    print(f'Streamlit app running at: {public_url}')
    # Make a request to the LocalTunnel URL with custom headers to bypass the password screen
    headers = {
        'User-Agent': 'Mozilla/5.0',
        'bypass-tunnel-reminder': '1'
    }
    response = requests.get(public_url, headers=headers)
    if response.status_code == 200:
        print(f'Access the Streamlit app at: {public_url}')
    else:
        print('Failed to bypass the password screen.')
else:
    print('Failed to create LocalTunnel.')

# Get the IP address for tunnel password
!wget -q -O - https://loca.lt/mytunnelpassword

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 79.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.1/871.1 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 703.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31529 sha256


added 22 packages, and audited 23 packages in 2s

3 packages are looking for funding
  run `npm fund` for details

1 moderate severity vulnerability

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
Cloning into 'slowfast'...
remote: Enumerating objects: 2149, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 2149 (delta 46), reused 59 (delta 35), pack-reused 2013 (from 1)
Receiving objects: 100% (2149/2149), 31.50 MiB | 20.77 MiB/s, done.
Resolving deltas: 100% (1457/1457), done.
/content/slowfast
running build
running build_py
creating build
creating build/lib
creating build/lib/slowfast
copying slowfast/__init__.py -> build/lib/slowfast
creating build/lib/slowfast/models
copying slowfast/models/masked.py -> build/lib/slowfast/models
copying slowfast/models/common.py -> build/lib/slowfast/models
copying slowfast/models/__init__.py -> build/lib/slowfast/m

NameError: name 'requests' is not defined

In [ ]:
!wget -q -O - https://loca.lt/mytunnelpassword

34.16.222.233

In [ ]:
# Install necessary Python dependencies
!pip install streamlit opencv-python-headless
!pip install ultralytics
!pip install simplejson
!pip install -U iopath
!pip install psutil
!pip install opencv-python
!pip install tensorboard
!pip install moviepy
!pip install scikit-learn  # Updated from sklearn to scikit-learn
!pip install pillow  # This installs PIL
!pip install pytorchvideo
!pip install 'git+https://github.com/facebookresearch/fairscale.git'

# Install PyAV and ffmpeg
!conda install av -c conda-forge -y
!conda install -c conda-forge ffmpeg -y

# Install dependencies for Detectron2
!pip install -U torch torchvision cython
!pip install -U 'git+https://github.com/facebookresearch/fvcore.git'
!pip install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

# Clone Detectron2 and install it
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo

# Install LocalTunnel globally using npm
!npm install -g localtunnel

# Clone the SlowFast repository and install it
!git clone https://github.com/facebookresearch/slowfast.git
%cd slowfast

# Add SlowFast to PYTHONPATH
import os
os.environ['PYTHONPATH'] += ":/content/slowfast"

# Build and develop SlowFast
!python setup.py build develop

# Ensure Python >= 3.8
!python --version

# Fix the pytorchvideo issue by cloning and installing from the local source
%cd /content
!git clone https://github.com/facebookresearch/pytorchvideo.git
%cd pytorchvideo
!pip install -e .

# Write the Streamlit app code to a file
app_code = """
import streamlit as st
import os
import shutil
import subprocess
import torch
import cv2
from PIL import Image
from ultralytics import YOLO
from torchvision import transforms
import time

# YOLOv8 Model Initialization
@st.cache_resource
def load_yolo_model():
    return YOLO('yolov8n.pt')

yolo_model = load_yolo_model()

# SlowFast Model Initialization using torch.hub
@st.cache_resource
def load_slowfast_model():
    model = torch.hub.load('facebookresearch/pytorchvideo', 'slowfast_r50', pretrained=True)
    model.eval()
    return model

slowfast_model = load_slowfast_model()

def process_video_with_yolo(video_path, chunk_size=50):
    st.write("Starting YOLOv8 inference...")
    cap = cv2.VideoCapture(video_path)
    all_detected_frames = []
    frame_count = 0

    while cap.isOpened():
        frames = []
        for _ in range(chunk_size):
            ret, frame = cap.read()
            if not ret:
                break
            frames.append(frame)
            frame_count += 1

        if not frames:
            break

        st.write(f"Processing frames {frame_count - len(frames) + 1} to {frame_count}...")
        results = yolo_model(frames, save=True, save_txt=True, conf=0.10, classes=67)

        for result in results:
            crops = result.crop(save=True)
            for crop in crops:
                img = Image.fromarray(crop["im"])
                all_detected_frames.append(img)

        st.write(f"Processed {frame_count} frames so far. Detected objects in {len(all_detected_frames)} frames.")

    cap.release()
    st.write(f"YOLOv8 inference completed. Total frames processed: {frame_count}")
    st.write(f"Total frames with detected objects: {len(all_detected_frames)}")
    return all_detected_frames

def detect_behavior_with_slowfast(frames, timeout=60):
    st.write("Starting SlowFast behavior analysis...")
    start_time = time.time()

    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225]),
    ])

    video_tensor = torch.stack([transform(frame) for frame in frames]).unsqueeze(0)

    if video_tensor.size(1) < 32:
        pad_size = 32 - video_tensor.size(1)
        video_tensor = torch.nn.functional.pad(video_tensor, (0, 0, 0, 0, 0, pad_size))

    video_tensor = video_tensor.permute(0, 2, 1, 3, 4)

    try:
        with torch.no_grad():
            output = slowfast_model(video_tensor)

        logits = output['logits']
        predicted_class = torch.argmax(logits, dim=1)
        behavior_detected = predicted_class.item() > 50

        st.write("SlowFast behavior analysis completed.")
        return behavior_detected

    except Exception as e:
        st.error(f"Error during SlowFast inference: {str(e)}")
        return False

    finally:
        elapsed_time = time.time() - start_time
        if elapsed_time > timeout:
            st.warning(f"SlowFast inference timed out after {timeout} seconds.")

def create_collage(frames, cols=5):
    if len(frames) == 0:
        return None
    frame_width, frame_height = frames[0].size
    rows = (len(frames) + cols - 1) // cols
    collage_width = cols * frame_width
    collage_height = rows * frame_height
    collage = Image.new('RGB', (collage_width, collage_height))
    for idx, frame in enumerate(frames):
        row = idx // cols
        col = idx % cols
        collage.paste(frame, (col * frame_width, row * frame_height))
    return collage

# Streamlit app
st.title('Driver Safety Detection: Cell Phones & Head Turning')

uploaded_file = st.file_uploader('Upload a video file', type=['mp4', 'avi', 'mov'])

if uploaded_file is not None:
    video_path = f'uploaded_video.{uploaded_file.type.split("/")[1]}'
    with open(video_path, "wb") as f:
        f.write(uploaded_file.getbuffer())

    st.video(video_path)
    st.write('Processing video...')

    try:
        detected_frames = process_video_with_yolo(video_path)
        if detected_frames:
            st.write(f'Detected {len(detected_frames)} frames with cell phones.')
            collage_image = create_collage(detected_frames)
            if collage_image:
                st.image(collage_image, caption='Collage of detected frames')

            st.write('Running SlowFast behavior analysis...')
            behavior_detected = detect_behavior_with_slowfast(detected_frames)
            if behavior_detected:
                st.write('Head turning behavior detected.')
                st.success('True Positive: Cell phone detected and head turning behavior detected.')
            else:
                st.write('No head turning behavior detected.')
                st.warning('False Positive: Cell phone detected but no head turning behavior detected.')
        else:
            st.write('No cell phones detected in the video.')
            st.info('No cell phones detected.')
    except Exception as e:
        st.error(f'Error during processing: {str(e)}')
"""

with open("app.py", "w") as file:
    file.write(app_code)
print("app.py file has been written.")

# Import subprocess and start the Streamlit app
import subprocess
subprocess.Popen(['streamlit', 'run', 'app.py'])

# Wait a few seconds for the Streamlit app to start
import time
time.sleep(5)

# Start LocalTunnel
localtunnel_process = subprocess.Popen(['lt', '--port', '8501'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Capture the LocalTunnel URL
public_url = None
for line in iter(localtunnel_process.stdout.readline, b''):
    decoded_line = line.decode().strip()
    print(decoded_line)
    if 'your url is:' in decoded_line:
        public_url = decoded_line.split(' ')[-1]
        break

if public_url:
    print(f'Streamlit app running at: {public_url}')
    # Make a request to the LocalTunnel URL with custom headers to bypass the password screen
    import requests
    headers = {
        'User-Agent': 'Mozilla/5.0',
        'bypass-tunnel-reminder': '1'
    }
    response = requests.get(public_url, headers=headers)
    if response.status_code == 200:
        print(f'Access the Streamlit app at: {public_url}')
    else:
        print('Failed to bypass the password screen.')
else:
    print('Failed to create LocalTunnel.')

# Get the IP address for tunnel password
!wget -q -O - https://loca.lt/mytunnelpassword

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.1/871.1 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31529 sha256=4


added 22 packages, and audited 23 packages in 2s

3 packages are looking for funding
  run `npm fund` for details

1 moderate severity vulnerability

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
Cloning into 'slowfast'...
remote: Enumerating objects: 2149, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (112/112), done.
remote: Total 2149 (delta 45), reused 70 (delta 24), pack-reused 2013 (from 1)
Receiving objects: 100% (2149/2149), 31.50 MiB | 10.84 MiB/s, done.
Resolving deltas: 100% (1455/1455), done.
/content/slowfast
running build
running build_py
creating build
creating build/lib
creating build/lib/slowfast
copying slowfast/__init__.py -> build/lib/slowfast
creating build/lib/slowfast/models
copying slowfast/models/masked.py -> build/lib/slowfast/models
copying slowfast/models/common.py -> build/lib/slowfast/models
copying slowfast/models/__init__.py -> build/lib/slowfast/m

In [ ]:
!wget -q -O - https://loca.lt/mytunnelpassword

34.142.132.101

In [ ]:
# Install necessary Python dependencies
!pip install streamlit opencv-python-headless
!pip install ultralytics
!pip install simplejson
!pip install -U iopath
!pip install psutil
!pip install opencv-python
!pip install tensorboard
!pip install moviepy
!pip install scikit-learn  # Updated from sklearn to scikit-learn
!pip install pillow  # This installs PIL
!pip install pytorchvideo
!pip install 'git+https://github.com/facebookresearch/fairscale.git'

# Install PyAV and ffmpeg
!conda install av -c conda-forge -y
!conda install -c conda-forge ffmpeg -y

# Install dependencies for Detectron2
!pip install -U torch torchvision cython
!pip install -U 'git+https://github.com/facebookresearch/fvcore.git'
!pip install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

# Clone Detectron2 and install it
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo

# Install LocalTunnel globally using npm
!npm install -g localtunnel

# Clone the SlowFast repository and install it
!git clone https://github.com/facebookresearch/slowfast.git
%cd slowfast

# Add SlowFast to PYTHONPATH
import os
os.environ['PYTHONPATH'] += ":/content/slowfast"

# Build and develop SlowFast
!python setup.py build develop

# Ensure Python >= 3.8
!python --version

# Fix the pytorchvideo issue by cloning and installing from the local source
%cd /content
!git clone https://github.com/facebookresearch/pytorchvideo.git
%cd pytorchvideo
!pip install -e .

# Write the Streamlit app code to a file
app_code = """
import streamlit as st
import os
import shutil
import subprocess
import torch
import cv2
from PIL import Image
from ultralytics import YOLO
from torchvision import transforms
import time

# YOLOv8 Model Initialization
@st.cache_resource
def load_yolo_model():
    return YOLO('yolov8n.pt')

yolo_model = load_yolo_model()

# SlowFast Model Initialization using torch.hub
@st.cache_resource
def load_slowfast_model():
    model = torch.hub.load('facebookresearch/pytorchvideo', 'slowfast_r50', pretrained=True)
    model.eval()
    return model

slowfast_model = load_slowfast_model()

def process_video_with_yolo(video_path, chunk_size=50):
    st.write("Starting YOLOv8 inference...")
    cap = cv2.VideoCapture(video_path)
    all_detected_frames = []
    frame_count = 0

    while cap.isOpened():
        frames = []
        for _ in range(chunk_size):
            ret, frame = cap.read()
            if not ret:
                break
            frames.append(frame)
            frame_count += 1

        if not frames:
            break

        st.write(f"Processing frames {frame_count - len(frames) + 1} to {frame_count}...")
        results = yolo_model(frames, save=True, save_txt=True, conf=0.10, classes=67)

        for result in results:
            if result.boxes:  # Check if there are any detections
                for box in result.boxes:
                    # Extract the bounding box coordinates
                    x1, y1, x2, y2 = box.xyxy[0].tolist()
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

                    # Crop the detected region from the original frame
                    cropped_img = Image.fromarray(result.orig_img[y1:y2, x1:x2])
                    all_detected_frames.append(cropped_img)

        st.write(f"Processed {frame_count} frames so far. Detected objects in {len(all_detected_frames)} frames.")

    cap.release()
    st.write(f"YOLOv8 inference completed. Total frames processed: {frame_count}")
    st.write(f"Total frames with detected objects: {len(all_detected_frames)}")
    return all_detected_frames

def detect_behavior_with_slowfast(frames, timeout=60):
    st.write("Starting SlowFast behavior analysis...")
    start_time = time.time()

    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225]),
    ])

    video_tensor = torch.stack([transform(frame) for frame in frames]).unsqueeze(0)

    if video_tensor.size(1) < 32:
        pad_size = 32 - video_tensor.size(1)
        video_tensor = torch.nn.functional.pad(video_tensor, (0, 0, 0, 0, 0, pad_size))

    video_tensor = video_tensor.permute(0, 2, 1, 3, 4)

    try:
        with torch.no_grad():
            output = slowfast_model(video_tensor)

        logits = output['logits']
        predicted_class = torch.argmax(logits, dim=1)
        behavior_detected = predicted_class.item() > 50

        st.write("SlowFast behavior analysis completed.")
        return behavior_detected

    except Exception as e:
        st.error(f"Error during SlowFast inference: {str(e)}")
        return False

    finally:
        elapsed_time = time.time() - start_time
        if elapsed_time > timeout:
            st.warning(f"SlowFast inference timed out after {timeout} seconds.")

def create_collage(frames, cols=5):
    if len(frames) == 0:
        return None
    frame_width, frame_height = frames[0].size
    rows = (len(frames) + cols - 1) // cols
    collage_width = cols * frame_width
    collage_height = rows * frame_height
    collage = Image.new('RGB', (collage_width, collage_height))
    for idx, frame in enumerate(frames):
        row = idx // cols
        col = idx % cols
        collage.paste(frame, (col * frame_width, row * frame_height))
    return collage

# Streamlit app
st.title('Driver Safety Detection: Cell Phones & Head Turning')

uploaded_file = st.file_uploader('Upload a video file', type=['mp4', 'avi', 'mov'])

if uploaded_file is not None:
    video_path = f'uploaded_video.{uploaded_file.type.split("/")[1]}'
    with open(video_path, "wb") as f:
        f.write(uploaded_file.getbuffer())

    st.video(video_path)
    st.write('Processing video...')

    try:
        detected_frames = process_video_with_yolo(video_path)
        if detected_frames:
            st.write(f'Detected {len(detected_frames)} frames with cell phones.')
            collage_image = create_collage(detected_frames)
            if collage_image:
                st.image(collage_image, caption='Collage of detected frames')

            st.write('Running SlowFast behavior analysis...')
            behavior_detected = detect_behavior_with_slowfast(detected_frames)
            if behavior_detected:
                st.write('Head turning behavior detected.')
                st.success('True Positive: Cell phone detected and head turning behavior detected.')
            else:
                st.write('No head turning behavior detected.')
                st.warning('False Positive: Cell phone detected but no head turning behavior detected.')
        else:
            st.write('No cell phones detected in the video.')
            st.info('No cell phones detected.')
    except Exception as e:
        st.error(f'Error during processing: {str(e)}')
"""

with open("app.py", "w") as file:
    file.write(app_code)
print("app.py file has been written.")

# Import subprocess and start the Streamlit app
import subprocess
subprocess.Popen(['streamlit', 'run', 'app.py'])

# Wait a few seconds for the Streamlit app to start
import time
time.sleep(5)

# Start LocalTunnel
localtunnel_process = subprocess.Popen(['lt', '--port', '8501'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Capture the LocalTunnel URL
public_url = None
for line in iter(localtunnel_process.stdout.readline, b''):
    decoded_line = line.decode().strip()
    print(decoded_line)
    if 'your url is:' in decoded_line:
        public_url = decoded_line.split(' ')[-1]
        break

if public_url:
    print(f'Streamlit app running at: {public_url}')
    # Make a request to the LocalTunnel URL with custom headers to bypass the password screen
    import requests
    headers = {
        'User-Agent': 'Mozilla/5.0',
        'bypass-tunnel-reminder': '1'
    }
    response = requests.get(public_url, headers=headers)
    if response.status_code == 200:
        print(f'Access the Streamlit app at: {public_url}')
    else:
        print('Failed to bypass the password screen.')
else:
    print('Failed to create LocalTunnel.')

# Get the IP address for tunnel password
!wget -q -O - https://loca.lt/mytunnelpassword

  Using cached iopath-0.1.10-py3-none-any.whl
  Attempting uninstall: iopath
    Found existing installation: iopath 0.1.9
    Uninstalling iopath-0.1.9:
      Successfully uninstalled iopath-0.1.9
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
detectron2 0.6 requires iopath<0.1.10,>=0.1.7, but you have iopath 0.1.10 which is incompatible.
  Cloning https://github.com/facebookresearch/fairscale.git to /tmp/pip-req-build-jmykwy6g
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/fairscale.git /tmp/pip-req-build-jmykwy6g
  Resolved https://github.com/facebookresearch/fairscale.git to commit 5f484b3545f27eddb19d970fbe1d361b9c5f2b07
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
/bin/bash: line 1: conda: command not found
/bin/bash: line

In [ ]:
# Install necessary Python dependencies
!pip install streamlit opencv-python-headless
!pip install ultralytics
!pip install simplejson
!pip install -U iopath
!pip install psutil
!pip install opencv-python
!pip install tensorboard
!pip install moviepy
!pip install scikit-learn  # Updated from sklearn to scikit-learn
!pip install pillow  # This installs PIL
!pip install pytorchvideo
!pip install 'git+https://github.com/facebookresearch/fairscale.git'

# Install PyAV and ffmpeg
!conda install av -c conda-forge -y
!conda install -c conda-forge ffmpeg -y

# Install dependencies for Detectron2
!pip install -U torch torchvision cython
!pip install -U 'git+https://github.com/facebookresearch/fvcore.git'
!pip install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

# Clone Detectron2 and install it
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo

# Install LocalTunnel globally using npm
!npm install -g localtunnel

# Clone the SlowFast repository and install it
!git clone https://github.com/facebookresearch/slowfast.git
%cd slowfast

# Add SlowFast to PYTHONPATH
import os
os.environ['PYTHONPATH'] += ":/content/slowfast"

# Build and develop SlowFast
!python setup.py build develop

# Ensure Python >= 3.8
!python --version

# Fix the pytorchvideo issue by cloning and installing from the local source
%cd /content
!git clone https://github.com/facebookresearch/pytorchvideo.git
%cd pytorchvideo
!pip install -e .

# Write the Streamlit app code to a file
app_code = """
import streamlit as st
import os
import shutil
import subprocess
import torch
import cv2
from PIL import Image
from ultralytics import YOLO
from torchvision import transforms
import time

# YOLOv8 Model Initialization
@st.cache_resource
def load_yolo_model():
    return YOLO('yolov8n.pt')

yolo_model = load_yolo_model()

# SlowFast Model Initialization using torch.hub
@st.cache_resource
def load_slowfast_model():
    model = torch.hub.load('facebookresearch/pytorchvideo', 'slowfast_r50', pretrained=True)
    model.eval()
    return model

slowfast_model = load_slowfast_model()

def process_video_with_yolo(video_path, chunk_size=50):
    st.write("Starting YOLOv8 inference...")
    cap = cv2.VideoCapture(video_path)
    all_detected_frames = []
    frame_count = 0

    while cap.isOpened():
        frames = []
        for _ in range(chunk_size):
            ret, frame = cap.read()
            if not ret:
                break
            frames.append(frame)
            frame_count += 1

        if not frames:
            break

        st.write(f"Processing frames {frame_count - len(frames) + 1} to {frame_count}...")
        results = yolo_model(frames, save=True, save_txt=True, conf=0.10, classes=67)

        for result in results:
            if result.boxes:  # Check if there are any detections
                for box in result.boxes:
                    # Extract the bounding box coordinates
                    x1, y1, x2, y2 = box.xyxy[0].tolist()
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

                    # Crop the detected region from the original frame
                    cropped_img = Image.fromarray(result.orig_img[y1:y2, x1:x2])
                    all_detected_frames.append(cropped_img)

        st.write(f"Processed {frame_count} frames so far. Detected objects in {len(all_detected_frames)} frames.")

    cap.release()
    st.write(f"YOLOv8 inference completed. Total frames processed: {frame_count}")
    st.write(f"Total frames with detected objects: {len(all_detected_frames)}")
    return all_detected_frames

def detect_behavior_with_slowfast(frames, timeout=60):
    st.write("Starting SlowFast behavior analysis...")
    start_time = time.time()

    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225]),
    ])

    video_tensor = torch.stack([transform(frame) for frame in frames]).unsqueeze(0)

    if video_tensor.size(1) < 32:
        pad_size = 32 - video_tensor.size(1)
        video_tensor = torch.nn.functional.pad(video_tensor, (0, 0, 0, 0, 0, pad_size))

    video_tensor = video_tensor.permute(0, 2, 1, 3, 4)

    try:
        with torch.no_grad():
            output = slowfast_model(video_tensor)

        logits = output['logits']
        predicted_class = torch.argmax(logits, dim=1)
        behavior_detected = predicted_class.item() > 50

        st.write("SlowFast behavior analysis completed.")
        return behavior_detected

    except Exception as e:
        st.error(f"Error during SlowFast inference: {str(e)}")
        return False

    finally:
        elapsed_time = time.time() - start_time
        if elapsed_time > timeout:
            st.warning(f"SlowFast inference timed out after {timeout} seconds.")

def create_collage(frames, cols=5):
    if len(frames) == 0:
        return None
    frame_width, frame_height = frames[0].size
    rows = (len(frames) + cols - 1) // cols
    collage_width = cols * frame_width
    collage_height = rows * frame_height
    collage = Image.new('RGB', (collage_width, collage_height))
    for idx, frame in enumerate(frames):
        row = idx // cols
        col = idx % cols
        collage.paste(frame, (col * frame_width, row * frame_height))
    return collage

# Streamlit app
st.title('Driver Safety Detection: Cell Phones & Head Turning')

uploaded_file = st.file_uploader('Upload a video file', type=['mp4', 'avi', 'mov'])

if uploaded_file is not None:
    video_path = f'uploaded_video.{uploaded_file.type.split("/")[1]}'
    with open(video_path, "wb") as f:
        f.write(uploaded_file.getbuffer())

    st.video(video_path)
    st.write('Processing video...')

    try:
        detected_frames = process_video_with_yolo(video_path)
        if detected_frames:
            st.write(f'Detected {len(detected_frames)} frames with cell phones.')
            collage_image = create_collage(detected_frames)
            if collage_image:
                st.image(collage_image, caption='Collage of detected frames')

            st.write('Running SlowFast behavior analysis...')
            behavior_detected = detect_behavior_with_slowfast(detected_frames)
            if behavior_detected:
                st.write('Head turning behavior detected.')
                st.success('True Positive: Cell phone detected and head turning behavior detected.')
            else:
                st.write('No head turning behavior detected.')
                st.warning('False Positive: Cell phone detected but no head turning behavior detected.')
        else:
            st.write('No cell phones detected in the video.')
            st.info('No cell phones detected.')
    except Exception as e:
        st.error(f'Error during processing: {str(e)}')
"""

with open("app.py", "w") as file:
    file.write(app_code)
print("app.py file has been written.")

# Import subprocess and start the Streamlit app
import subprocess
subprocess.Popen(['streamlit', 'run', 'app.py'])

# Wait a few seconds for the Streamlit app to start
import time
time.sleep(5)

# Start LocalTunnel
localtunnel_process = subprocess.Popen(['lt', '--port', '8501'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Capture the LocalTunnel URL
public_url = None
for line in iter(localtunnel_process.stdout.readline, b''):
    decoded_line = line.decode().strip()
    print(decoded_line)
    if 'your url is:' in decoded_line:
        public_url = decoded_line.split(' ')[-1]
        break

if public_url:
    print(f'Streamlit app running at: {public_url}')
    # Make a request to the LocalTunnel URL with custom headers to bypass the password screen
    import requests
    headers = {
        'User-Agent': 'Mozilla/5.0',
        'bypass-tunnel-reminder': '1'
    }
    response = requests.get(public_url, headers=headers)
    if response.status_code == 200:
        print(f'Access the Streamlit app at: {public_url}')
    else:
        print('Failed to bypass the password screen.')
else:
    print('Failed to create LocalTunnel.')

# Get the IP address for tunnel password
!wget -q -O - https://loca.lt/mytunnelpassword

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.1/871.1 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31529 sha256=7


added 22 packages, and audited 23 packages in 2s

3 packages are looking for funding
  run `npm fund` for details

1 moderate severity vulnerability

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
Cloning into 'slowfast'...
remote: Enumerating objects: 2149, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (112/112), done.
remote: Total 2149 (delta 45), reused 70 (delta 24), pack-reused 2013 (from 1)
Receiving objects: 100% (2149/2149), 31.50 MiB | 40.36 MiB/s, done.
Resolving deltas: 100% (1455/1455), done.
/content/slowfast
running build
running build_py
creating build
creating build/lib
creating build/lib/slowfast
copying slowfast/__init__.py -> build/lib/slowfast
creating build/lib/slowfast/models
copying slowfast/models/masked.py -> build/lib/slowfast/models
copying slowfast/models/common.py -> build/lib/slowfast/models
copying slowfast/models/__init__.py -> build/lib/slowfast/m

In [ ]:
# Install necessary Python dependencies
!pip install streamlit opencv-python-headless
!pip install ultralytics
!pip install simplejson
!pip install -U iopath
!pip install psutil
!pip install opencv-python
!pip install tensorboard
!pip install moviepy
!pip install scikit-learn  # Updated from sklearn to scikit-learn
!pip install pillow  # This installs PIL
!pip install pytorchvideo
!pip install 'git+https://github.com/facebookresearch/fairscale.git'

# Install PyAV and ffmpeg
!conda install av -c conda-forge -y
!conda install -c conda-forge ffmpeg -y

# Install dependencies for Detectron2
!pip install -U torch torchvision cython
!pip install -U 'git+https://github.com/facebookresearch/fvcore.git'
!pip install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

# Clone Detectron2 and install it
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo

# Install LocalTunnel globally using npm
!npm install -g localtunnel

# Clone the SlowFast repository and install it
!git clone https://github.com/facebookresearch/slowfast.git
%cd slowfast

# Add SlowFast to PYTHONPATH
import os
os.environ['PYTHONPATH'] += ":/content/slowfast"

# Build and develop SlowFast
!python setup.py build develop

# Ensure Python >= 3.8
!python --version

# Fix the pytorchvideo issue by cloning and installing from the local source
%cd /content
!git clone https://github.com/facebookresearch/pytorchvideo.git
%cd pytorchvideo
!pip install -e .

# Write the Streamlit app code to a file
app_code = """
import streamlit as st
import os
import shutil
import subprocess
import torch
import cv2
from PIL import Image
from ultralytics import YOLO
from torchvision import transforms
import time

# YOLOv8 Model Initialization
@st.cache_resource
def load_yolo_model():
    return YOLO('yolov8n.pt')

yolo_model = load_yolo_model()

# SlowFast Model Initialization using torch.hub
@st.cache_resource
def load_slowfast_model():
    model = torch.hub.load('facebookresearch/pytorchvideo', 'slowfast_r50', pretrained=True)
    model.eval()
    return model

slowfast_model = load_slowfast_model()

def process_video_with_yolo(video_path, chunk_size=50):
    st.write("Starting YOLOv8 inference...")
    cap = cv2.VideoCapture(video_path)
    all_detected_frames = []
    frame_count = 0

    while cap.isOpened():
        frames = []
        for _ in range(chunk_size):
            ret, frame = cap.read()
            if not ret:
                break
            frames.append(frame)
            frame_count += 1

        if not frames:
            break

        st.write(f"Processing frames {frame_count - len(frames) + 1} to {frame_count}...")
        results = yolo_model(frames, save=True, save_txt=True, conf=0.10, classes=67)

        for result in results:
            if result.boxes:  # Check if there are any detections
                for box in result.boxes:
                    # Extract the bounding box coordinates
                    x1, y1, x2, y2 = box.xyxy[0].tolist()
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

                    # Crop the detected region from the original frame
                    cropped_img = Image.fromarray(result.orig_img[y1:y2, x1:x2])
                    all_detected_frames.append(cropped_img)

        st.write(f"Processed {frame_count} frames so far. Detected objects in {len(all_detected_frames)} frames.")

    cap.release()
    st.write(f"YOLOv8 inference completed. Total frames processed: {frame_count}")
    st.write(f"Total frames with detected objects: {len(all_detected_frames)}")
    return all_detected_frames

def detect_behavior_with_slowfast(frames, timeout=60):
    st.write("Starting SlowFast behavior analysis...")
    start_time = time.time()

    # Ensure we have at least 32 frames
    if len(frames) < 32:
        st.warning(f"Not enough frames for SlowFast analysis. Need at least 32, but got {len(frames)}.")
        return False

    # Select 32 evenly spaced frames if we have more than 32
    if len(frames) > 32:
        frame_indices = np.linspace(0, len(frames) - 1, 32, dtype=int)
        frames = [frames[i] for i in frame_indices]

    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225]),
    ])

    # Prepare the slow and fast pathways
    slow_pathway = torch.stack([transform(frame) for frame in frames[::4]])  # Every 4th frame
    fast_pathway = torch.stack([transform(frame) for frame in frames])

    # Add batch dimension
    slow_pathway = slow_pathway.unsqueeze(0)
    fast_pathway = fast_pathway.unsqueeze(0)

    # Prepare input in the format expected by SlowFast
    input_frames = [slow_pathway, fast_pathway]

    try:
        with torch.no_grad():
            output = slowfast_model(input_frames)

        logits = output['logits']
        predicted_class = torch.argmax(logits, dim=1)
        behavior_detected = predicted_class.item() > 50  # You may need to adjust this threshold

        st.write("SlowFast behavior analysis completed.")
        return behavior_detected

    except Exception as e:
        st.error(f"Error during SlowFast inference: {str(e)}")
        return False

    finally:
        elapsed_time = time.time() - start_time
        if elapsed_time > timeout:
            st.warning(f"SlowFast inference timed out after {timeout} seconds.")

def create_collage(frames, cols=5):
    if len(frames) == 0:
        return None
    frame_width, frame_height = frames[0].size
    rows = (len(frames) + cols - 1) // cols
    collage_width = cols * frame_width
    collage_height = rows * frame_height
    collage = Image.new('RGB', (collage_width, collage_height))
    for idx, frame in enumerate(frames):
        row = idx // cols
        col = idx % cols
        collage.paste(frame, (col * frame_width, row * frame_height))
    return collage

# Streamlit app
st.title('Driver Safety Detection: Cell Phones & Head Turning')

uploaded_file = st.file_uploader('Upload a video file', type=['mp4', 'avi', 'mov'])

if uploaded_file is not None:
    video_path = f'uploaded_video.{uploaded_file.type.split("/")[1]}'
    with open(video_path, "wb") as f:
        f.write(uploaded_file.getbuffer())

    st.video(video_path)
    st.write('Processing video...')

    try:
        detected_frames = process_video_with_yolo(video_path)
        if detected_frames:
            st.write(f'Detected {len(detected_frames)} frames with cell phones.')
            collage_image = create_collage(detected_frames)
            if collage_image:
                st.image(collage_image, caption='Collage of detected frames')

            st.write('Running SlowFast behavior analysis...')
            behavior_detected = detect_behavior_with_slowfast(detected_frames)
            if behavior_detected:
                st.write('Head turning behavior detected.')
                st.success('True Positive: Cell phone detected and head turning behavior detected.')
            else:
                st.write('No head turning behavior detected.')
                st.warning('False Positive: Cell phone detected but no head turning behavior detected.')
        else:
            st.write('No cell phones detected in the video.')
            st.info('No cell phones detected.')
    except Exception as e:
        st.error(f'Error during processing: {str(e)}')
"""

with open("app.py", "w") as file:
    file.write(app_code)
print("app.py file has been written.")

# Import subprocess and start the Streamlit app
import subprocess
subprocess.Popen(['streamlit', 'run', 'app.py'])

# Wait a few seconds for the Streamlit app to start
import time
time.sleep(5)

# Start LocalTunnel
localtunnel_process = subprocess.Popen(['lt', '--port', '8501'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Capture the LocalTunnel URL
public_url = None
for line in iter(localtunnel_process.stdout.readline, b''):
    decoded_line = line.decode().strip()
    print(decoded_line)
    if 'your url is:' in decoded_line:
        public_url = decoded_line.split(' ')[-1]
        break

if public_url:
    print(f'Streamlit app running at: {public_url}')
    # Make a request to the LocalTunnel URL with custom headers to bypass the password screen
    import requests
    headers = {
        'User-Agent': 'Mozilla/5.0',
        'bypass-tunnel-reminder': '1'
    }
    response = requests.get(public_url, headers=headers)
    if response.status_code == 200:
        print(f'Access the Streamlit app at: {public_url}')
    else:
        print('Failed to bypass the password screen.')
else:
    print('Failed to create LocalTunnel.')

# Get the IP address for tunnel password
!wget -q -O - https://loca.lt/mytunnelpassword

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 95.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.1/871.1 kB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31529 sha256=


added 22 packages, and audited 23 packages in 2s

3 packages are looking for funding
  run `npm fund` for details

1 moderate severity vulnerability

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
Cloning into 'slowfast'...
remote: Enumerating objects: 2149, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 2149 (delta 46), reused 59 (delta 35), pack-reused 2013 (from 1)
Receiving objects: 100% (2149/2149), 31.51 MiB | 17.22 MiB/s, done.
Resolving deltas: 100% (1457/1457), done.
/content/slowfast
running build
running build_py
creating build
creating build/lib
creating build/lib/slowfast
copying slowfast/__init__.py -> build/lib/slowfast
creating build/lib/slowfast/models
copying slowfast/models/resnet_helper.py -> build/lib/slowfast/models
copying slowfast/models/attention.py -> build/lib/slowfast/models
copying slowfast/models/optimizer.py -> build/lib

In [ ]:
# Install necessary Python dependencies
!pip install streamlit opencv-python-headless
!pip install ultralytics
!pip install simplejson
!pip install -U iopath
!pip install psutil
!pip install opencv-python
!pip install tensorboard
!pip install moviepy
!pip install scikit-learn  # Updated from sklearn to scikit-learn
!pip install pillow  # This installs PIL
!pip install pytorchvideo
!pip install 'git+https://github.com/facebookresearch/fairscale.git'

# Install PyAV and ffmpeg
!conda install av -c conda-forge -y
!conda install -c conda-forge ffmpeg -y

# Install dependencies for Detectron2
!pip install -U torch torchvision cython
!pip install -U 'git+https://github.com/facebookresearch/fvcore.git'
!pip install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

# Clone Detectron2 and install it
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo

# Install LocalTunnel globally using npm
!npm install -g localtunnel

# Clone the SlowFast repository and install it
!git clone https://github.com/facebookresearch/slowfast.git
%cd slowfast

# Add SlowFast to PYTHONPATH
import os
os.environ['PYTHONPATH'] += ":/content/slowfast"

# Build and develop SlowFast
!python setup.py build develop

# Ensure Python >= 3.8
!python --version

# Fix the pytorchvideo issue by cloning and installing from the local source
%cd /content
!git clone https://github.com/facebookresearch/pytorchvideo.git
%cd pytorchvideo
!pip install -e .

# Write the Streamlit app code to a file
app_code = """
import streamlit as st
import os
import shutil
import subprocess
import torch
import cv2
from PIL import Image
from ultralytics import YOLO
from torchvision import transforms
import time

# YOLOv8 Model Initialization
@st.cache_resource
def load_yolo_model():
    return YOLO('yolov8n.pt')

yolo_model = load_yolo_model()

# SlowFast Model Initialization using torch.hub
@st.cache_resource
def load_slowfast_model():
    model = torch.hub.load('facebookresearch/pytorchvideo', 'slowfast_r50', pretrained=True)
    model.eval()
    return model

slowfast_model = load_slowfast_model()

def process_video_with_yolo(video_path, chunk_size=50):
    st.write("Starting YOLOv8 inference...")
    cap = cv2.VideoCapture(video_path)
    all_detected_frames = []
    frame_count = 0

    while cap.isOpened():
        frames = []
        for _ in range(chunk_size):
            ret, frame = cap.read()
            if not ret:
                break
            frames.append(frame)
            frame_count += 1

        if not frames:
            break

        st.write(f"Processing frames {frame_count - len(frames) + 1} to {frame_count}...")
        results = yolo_model(frames, save=True, save_txt=True, conf=0.10, classes=67)

        for result in results:
            if result.boxes:  # Check if there are any detections
                for box in result.boxes:
                    # Extract the bounding box coordinates
                    x1, y1, x2, y2 = box.xyxy[0].tolist()
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

                    # Crop the detected region from the original frame
                    cropped_img = Image.fromarray(result.orig_img[y1:y2, x1:x2])
                    all_detected_frames.append(cropped_img)

        st.write(f"Processed {frame_count} frames so far. Detected objects in {len(all_detected_frames)} frames.")

    cap.release()
    st.write(f"YOLOv8 inference completed. Total frames processed: {frame_count}")
    st.write(f"Total frames with detected objects: {len(all_detected_frames)}")
    return all_detected_frames

def detect_behavior_with_slowfast(frames, timeout=60):
    st.write("Starting SlowFast behavior analysis...")
    start_time = time.time()

    # Ensure we have at least 32 frames by padding if necessary
    if len(frames) < 32:
        padding_frames = [frames[-1]] * (32 - len(frames))  # Repeat the last frame
        frames.extend(padding_frames)


    # Select 32 evenly spaced frames if we have more than 32
    if len(frames) > 32:
        frame_indices = np.linspace(0, len(frames) - 1, 32, dtype=int)
        frames = [frames[i] for i in frame_indices]

    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225]),
    ])

    # Prepare the slow and fast pathways
    slow_pathway = torch.stack([transform(frame) for frame in frames[::4]])  # Every 4th frame
    fast_pathway = torch.stack([transform(frame) for frame in frames])

    # Add batch dimension
    slow_pathway = slow_pathway.unsqueeze(0)
    fast_pathway = fast_pathway.unsqueeze(0)

    # Prepare input in the format expected by SlowFast
    input_frames = [slow_pathway, fast_pathway]

    try:
        with torch.no_grad():
            output = slowfast_model(input_frames)

        logits = output['logits']
        predicted_class = torch.argmax(logits, dim=1)
        behavior_detected = predicted_class.item() > 50  # You may need to adjust this threshold

        st.write("SlowFast behavior analysis completed.")
        return behavior_detected

    except Exception as e:
        st.error(f"Error during SlowFast inference: {str(e)}")
        return False

    finally:
        elapsed_time = time.time() - start_time
        if elapsed_time > timeout:
            st.warning(f"SlowFast inference timed out after {timeout} seconds.")

def create_collage(frames, cols=5):
    if len(frames) == 0:
        return None
    frame_width, frame_height = frames[0].size
    rows = (len(frames) + cols - 1) // cols
    collage_width = cols * frame_width
    collage_height = rows * frame_height
    collage = Image.new('RGB', (collage_width, collage_height))
    for idx, frame in enumerate(frames):
        row = idx // cols
        col = idx % cols
        collage.paste(frame, (col * frame_width, row * frame_height))
    return collage

# Streamlit app
st.title('Driver Safety Detection: Cell Phones & Head Turning')

uploaded_file = st.file_uploader('Upload a video file', type=['mp4', 'avi', 'mov'])

if uploaded_file is not None:
    video_path = f'uploaded_video.{uploaded_file.type.split("/")[1]}'
    with open(video_path, "wb") as f:
        f.write(uploaded_file.getbuffer())

    st.video(video_path)
    st.write('Processing video...')

    try:
        detected_frames = process_video_with_yolo(video_path)
        if detected_frames:
            st.write(f'Detected {len(detected_frames)} frames with cell phones.')
            collage_image = create_collage(detected_frames)
            if collage_image:
                st.image(collage_image, caption='Collage of detected frames')

            st.write('Running SlowFast behavior analysis...')
            behavior_detected = detect_behavior_with_slowfast(detected_frames)
            if behavior_detected:
                st.write('Head turning behavior detected.')
                st.success('True Positive: Cell phone detected and head turning behavior detected.')
            else:
                st.write('No head turning behavior detected.')
                st.warning('False Positive: Cell phone detected but no head turning behavior detected.')
        else:
            st.write('No cell phones detected in the video.')
            st.info('No cell phones detected.')
    except Exception as e:
        st.error(f'Error during processing: {str(e)}')
"""

with open("app.py", "w") as file:
    file.write(app_code)
print("app.py file has been written.")

# Import subprocess and start the Streamlit app
import subprocess
subprocess.Popen(['streamlit', 'run', 'app.py'])

# Wait a few seconds for the Streamlit app to start
import time
time.sleep(5)

# Start LocalTunnel
localtunnel_process = subprocess.Popen(['lt', '--port', '8501'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Capture the LocalTunnel URL
public_url = None
for line in iter(localtunnel_process.stdout.readline, b''):
    decoded_line = line.decode().strip()
    print(decoded_line)
    if 'your url is:' in decoded_line:
        public_url = decoded_line.split(' ')[-1]
        break

if public_url:
    print(f'Streamlit app running at: {public_url}')
    # Make a request to the LocalTunnel URL with custom headers to bypass the password screen
    import requests
    headers = {
        'User-Agent': 'Mozilla/5.0',
        'bypass-tunnel-reminder': '1'
    }
    response = requests.get(public_url, headers=headers)
    if response.status_code == 200:
        print(f'Access the Streamlit app at: {public_url}')
    else:
        print('Failed to bypass the password screen.')
else:
    print('Failed to create LocalTunnel.')

# Get the IP address for tunnel password
!wget -q -O - https://loca.lt/mytunnelpassword

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 81.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.1/871.1 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31529 sha256=a


added 22 packages, and audited 23 packages in 2s

3 packages are looking for funding
  run `npm fund` for details

1 moderate severity vulnerability

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
Cloning into 'slowfast'...
remote: Enumerating objects: 2149, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (112/112), done.
remote: Total 2149 (delta 45), reused 70 (delta 24), pack-reused 2013 (from 1)
Receiving objects: 100% (2149/2149), 31.50 MiB | 19.99 MiB/s, done.
Resolving deltas: 100% (1455/1455), done.
/content/slowfast
running build
running build_py
creating build
creating build/lib
creating build/lib/slowfast
copying slowfast/__init__.py -> build/lib/slowfast
creating build/lib/slowfast/models
copying slowfast/models/resnet_helper.py -> build/lib/slowfast/models
copying slowfast/models/attention.py -> build/lib/slowfast/models
copying slowfast/models/optimizer.py -> build/lib

In [ ]:
import torch
from pytorchvideo.models.hub import slowfast_r50  # Import the SlowFast model
import torchvision.transforms as transforms
from pytorchvideo.transforms import ShortSideScale, UniformTemporalSubsample
from pytorchvideo.data.encoded_video import EncodedVideo
import torch.nn.functional as F
import cv2

# Load the pre-trained SlowFast model
model = slowfast_r50(pretrained=True)
model = model.eval()  # Set the model to evaluation mode

# Load the video using PyTorchVideo's EncodedVideo class
video_path = "/content/head_turning_2.mp4"
video = EncodedVideo.from_path(video_path)

# Select the time range to sample from the video (e.g., from 0 to 8 seconds)
start_time = 0
end_time = 8  # In seconds

# Sample frames uniformly
video_data = video.get_clip(start_sec=start_time, end_sec=end_time)
frames = video_data["video"]  # [T, C, H, W] - this should be the shape after this step

# Define the transforms to apply to each frame
transform = transforms.Compose([
    UniformTemporalSubsample(32),  # Sample 32 frames uniformly
    ShortSideScale(256),
    transforms.Lambda(lambda x: x / 255.0),  # Normalize to [0, 1]
])

# Apply the initial transforms
frames = transform(frames)  # [T, C, H, W] format is preserved

# Now normalize the RGB channels
normalize = transforms.Normalize(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225])
frames = normalize(frames)  # Ensure [T, C, H, W] - the shape should now be [32, 3, 256, 256]

# Verify the number of channels
print(f"Number of channels after transformation: {frames.shape[1]}")  # Should be 3

# Prepare the input format for SlowFast
fast_pathway = frames.permute(1, 0, 2, 3)  # [C, T, H, W]
slow_pathway = frames.permute(1, 0, 2, 3)[:, :, ::4, :]  # [C, T//4, H, W]

# Combine pathways into a list and add a batch dimension
inputs = [slow_pathway.unsqueeze(0), fast_pathway.unsqueeze(0)]

# Run inference
with torch.no_grad():
    preds = model(inputs)

# Take the top-1 prediction
top1_pred = torch.argmax(preds, dim=1)
print(f"Predicted class: {top1_pred.item()}")


RuntimeError: The size of tensor a (32) must match the size of tensor b (3) at non-singleton dimension 1

In [ ]:
import torch
from pytorchvideo.models.hub import slowfast_r50
import torchvision.transforms as transforms
from pytorchvideo.transforms import ShortSideScale, UniformTemporalSubsample
from pytorchvideo.data.encoded_video import EncodedVideo
import torch.nn.functional as F
import cv2

# Load the pre-trained SlowFast model
model = slowfast_r50(pretrained=True)
model = model.eval()

# Load the video using PyTorchVideo's EncodedVideo class
video_path = "/content/head_turning_2.mp4"
video = EncodedVideo.from_path(video_path)

# Select the time range to sample from the video (e.g., from 0 to 8 seconds)
start_time = 0
end_time = 8  # In seconds

# Sample frames uniformly
video_data = video.get_clip(start_sec=start_time, end_sec=end_time)
frames = video_data["video"]  # [T, C, H, W] - this should be the shape after this step

# Define the transforms to apply to each frame
transform = transforms.Compose([
    UniformTemporalSubsample(32),  # Sample 32 frames uniformly
    ShortSideScale(256),
    transforms.CenterCrop(224),  # Add center crop to get 224x224
    transforms.Lambda(lambda x: x / 255.0),  # Normalize to [0, 1]
])

# Apply the initial transforms
frames = transform(frames)  # [T, C, H, W] format is preserved

# Now normalize the RGB channels
normalize = transforms.Normalize(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225])
frames = frames.permute(1, 0, 2, 3)  # Change to [C, T, H, W]
frames = normalize(frames)
frames = frames.permute(1, 0, 2, 3)  # Change back to [T, C, H, W]

# Verify the number of channels
print(f"Shape after transformation: {frames.shape}")  # Should be [32, 3, 224, 224]

# Prepare the input format for SlowFast
fast_pathway = frames
slow_pathway = torch.index_select(
    frames,
    1,
    torch.linspace(0, frames.shape[1] - 1, frames.shape[1] // 4).long(),
)

# Combine pathways into a list and add a batch dimension
inputs = [slow_pathway.unsqueeze(0), fast_pathway.unsqueeze(0)]

# Run inference
with torch.no_grad():
    preds = model(inputs)

# Take the top-1 prediction
top1_pred = torch.argmax(preds, dim=1)
print(f"Predicted class: {top1_pred.item()}")

Shape after transformation: torch.Size([3, 32, 224, 224])
Predicted class: 219


In [ ]:
import torch
from pytorchvideo.models.hub import slowfast_r50
import torchvision.transforms as transforms
from pytorchvideo.transforms import ShortSideScale, UniformTemporalSubsample
from pytorchvideo.data.encoded_video import EncodedVideo
import torch.nn.functional as F
import cv2

# Load the pre-trained SlowFast model
model = slowfast_r50(pretrained=True)
model = model.eval()

# Load the video using PyTorchVideo's EncodedVideo class
video_path = "/content/head_turning_2.mp4"
video = EncodedVideo.from_path(video_path)

# Select the time range to sample from the video (e.g., from 0 to 8 seconds)
start_time = 0
end_time = 8  # In seconds

# Sample frames uniformly
video_data = video.get_clip(start_sec=start_time, end_sec=end_time)
frames = video_data["video"]  # [T, C, H, W] - this should be the shape after this step

# Define the transforms to apply to each frame
transform = transforms.Compose([
    UniformTemporalSubsample(32),  # Sample 32 frames uniformly
    ShortSideScale(256),
    transforms.CenterCrop(224),  # Add center crop to get 224x224
    transforms.Lambda(lambda x: x / 255.0),  # Normalize to [0, 1]
])

# Apply the initial transforms
frames = transform(frames)  # [T, C, H, W] format is preserved

# Now normalize the RGB channels
normalize = transforms.Normalize(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225])
frames = normalize(frames)

# Verify the number of channels
print(f"Shape after transformation: {frames.shape}")  # Should be [32, 3, 224, 224]

# Prepare the input format for SlowFast
fast_pathway = frames
slow_pathway = torch.index_select(
    frames,
    1,
    torch.linspace(0, frames.shape[1] - 1, frames.shape[1] // 4).long(),
)

# Combine pathways into a list and add a batch dimension
inputs = [slow_pathway.unsqueeze(0), fast_pathway.unsqueeze(0)]

# Run inference
with torch.no_grad():
    preds = model(inputs)

# Take the top-1 prediction
top1_pred = torch.argmax(preds, dim=1)
print(f"Predicted class index: {top1_pred.item()}")

# Load Kinetics-400 class labels
kinetics_labels = []
with open('kinetics_400_labels.txt', 'r') as f:
    for line in f:
        kinetics_labels.append(line.strip())

# Get the predicted class name
predicted_class_name = kinetics_labels[top1_pred.item()]
print(f"Predicted action: {predicted_class_name}")

# Print top 5 predictions
top5_pred = torch.topk(preds, k=5)
print("\nTop 5 predictions:")
for i in range(5):
    print(f"{i+1}. {kinetics_labels[top5_pred.indices[0][i]]}: {top5_pred.values[0][i]:.3f}")

RuntimeError: The size of tensor a (32) must match the size of tensor b (3) at non-singleton dimension 1

In [ ]:

!pip install pytorchvideo
!pip install torchvision
!pip install iopath
!pip install torch
!pip install -U 'git+https://github.com/facebookresearch/fvcore.git
!pip install torch torchvision --upgrade
!pip install -e git+https://github.com/facebookresearch/pytorchvideo.git#egg=pytorchvideo
!pip install iopath psutil moviepy scikit-learn pillow
!pip install 'git+https://github.com/facebookresearch/fairscale.git'


# Clone the SlowFast repository and install it
!git clone https://github.com/facebookresearch/slowfast.git
%cd slowfast

# Add SlowFast to PYTHONPATH
import os
os.environ['PYTHONPATH'] += ":/content/slowfast"

# Build and develop SlowFast
!python setup.py build develop




# Remove any existing pytorchvideo installations
!pip uninstall -y pytorchvideo

# Clone the latest version of the PytorchVideo repository
!git clone https://github.com/facebookresearch/pytorchvideo.git

# Install PytorchVideo from the cloned repository
%cd pytorchvideo
!pip install -e .

import torch
from pytorchvideo.models.hub import slowfast_r50
from pytorchvideo.models.hub import slowfast_r101
import torchvision.transforms as transforms
from pytorchvideo.transforms import ShortSideScale, UniformTemporalSubsample
from pytorchvideo.data.encoded_video import EncodedVideo
import torch.nn.functional as F
import cv2

# Load the pre-trained SlowFast model
model = slowfast_r101(pretrained="kinetics_600")
model = model.eval()

# Load the video using PyTorchVideo's EncodedVideo class
video_path = "/content/texting_2.mp4"
video = EncodedVideo.from_path(video_path)

# Select the time range to sample from the video (e.g., from 0 to 8 seconds)
start_time = 0
end_time = 8  # In seconds

# Sample frames uniformly
video_data = video.get_clip(start_sec=start_time, end_sec=end_time)
frames = video_data["video"]  # [T, C, H, W] - this should be the shape after this step

# Define the transforms to apply to each frame
transform = transforms.Compose([
    UniformTemporalSubsample(32),  # Sample 32 frames uniformly
    ShortSideScale(256),
    transforms.CenterCrop(224),  # Add center crop to get 224x224
    transforms.Lambda(lambda x: x / 255.0),  # Normalize to [0, 1]
])

# Apply the initial transforms
frames = transform(frames)  # [T, C, H, W] format is preserved

# Now normalize the RGB channels
normalize = transforms.Normalize(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225])
frames = frames.permute(1, 0, 2, 3)  # Change to [C, T, H, W]
frames = normalize(frames)
frames = frames.permute(1, 0, 2, 3)  # Change back to [T, C, H, W]

# Verify the number of channels
print(f"Shape after transformation: {frames.shape}")  # Should be [32, 3, 224, 224]

# Prepare the input format for SlowFast
fast_pathway = frames
slow_pathway = torch.index_select(
    frames,
    1,
    torch.linspace(0, frames.shape[1] - 1, frames.shape[1] // 4).long(),
)

# Combine pathways into a list and add a batch dimension
inputs = [slow_pathway.unsqueeze(0), fast_pathway.unsqueeze(0)]

# Run inference
with torch.no_grad():
    preds = model(inputs)

# Take the top-1 prediction
top1_pred = torch.argmax(preds, dim=1)
print(f"Predicted class: {top1_pred.item()}")

# Load Kinetics-400 class labels
kinetics_labels = []
with open('/content/label_map.txt', 'r') as f:
    for line in f:
        kinetics_labels.append(line.strip())

# Get the predicted class name
predicted_class_name = kinetics_labels[top1_pred.item()]
print(f"Predicted action: {predicted_class_name}")

# Print top 5 predictions
top5_pred = torch.topk(preds, k=5)
print("\nTop 5 predictions:")
for i in range(5):
    print(f"{i+1}. {kinetics_labels[top5_pred.indices[0][i]]}: {top5_pred.values[0][i]:.3f}")

/bin/bash: -c: line 1: unexpected EOF while looking for matching `''
/bin/bash: -c: line 2: syntax error: unexpected end of file
Obtaining pytorchvideo from git+https://github.com/facebookresearch/pytorchvideo.git#egg=pytorchvideo
  Cloning https://github.com/facebookresearch/pytorchvideo.git to ./src/pytorchvideo
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/pytorchvideo.git /content/slowfast/pytorchvideo/slowfast/pytorchvideo/slowfast/pytorchvideo/slowfast/pytorchvideo/slowfast/pytorchvideo/slowfast/pytorchvideo/slowfast/pytorchvideo/slowfast/pytorchvideo/slowfast/pytorchvideo/slowfast/pytorchvideo/slowfast/pytorchvideo/slowfast/pytorchvideo/src/pytorchvideo
  Resolved https://github.com/facebookresearch/pytorchvideo.git to commit e02c17b480f4b2a70a000df4e1aaf9d45fdaefb5
  Preparing metadata (setup.py) ... done
  Attempting uninstall: pytorchvideo
    Found existing installation: pytorchvideo 0.1.5
    Uninstalling pytorchvideo-0.1.5:
    